scripts for general pca analysis of data

In [170]:
### load arrs, pgv4 new

import jax
import jax.numpy as jnp
from jax import jit
import jax.random as rnd
import numpy as np

np.set_printoptions(threshold=1000, linewidth=200)
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.gridspec as gridspec
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.patches import Ellipse
import os
import functools

def load_large_outputs(str_):
    path_ = os.path.dirname(os.getcwd()) + '/sc_project/large_outputs/' # '/sc_project/test_data/' # Path(__file__).resolve().parents[1]
    with open(path_+str_,'rb') as file_:
        # param = pickle.load(file_)
        param_ = jnp.load(file_,allow_pickle=True)
    return param_

def freeze_array(BA, P):
    P2 = np.zeros_like(P)
    P2[:, 0] = P[:, 0]
    for t in range(1, P.shape[1]):
        if BA[t] == 0:
            P2[:, t] = P[:, t]  # Update to current value in P
        else:
            P2[:, t] = P2[:, t - 1]  # Hold previous value
    return P2

def compare_arrays(pos_arr, pos_plan_arr):
    return np.where(pos_arr == pos_plan_arr, 0, 1)

file_ = 'outer_loop_pg_new_v4f_ttg_26_11-115931.pkl' # 'outer_loop_pg_new_v4f_test_02_11-183526.pkl' # 'outer_loop_pg_new_v4f_23_10-231650.pkl'

other = load_large_outputs(file_)

## if not test:
(selected_other,plan_info) = other
(r_arr,rp_arr,sample_arr,mask_arr,pos_plan_arr,pos_arr,dot_arr,policy_arr,hs_arr,hv_arr,vec_ind_arr,act_ind_arr),_ = selected_other
# plan:
# (r_arr,rp_arr,sample_arr,mask_arr,pos_plan_arr,pos_arr,dot_arr,policy_arr,hs_arr,hv_arr,vec_ind_arr,act_ind_arr),_ = plan_info['other_']

# if test:
# (r_arr,rp_arr,sample_arr,mask_arr,pos_plan_arr,pos_arr,dot_arr,policy_arr,hs_arr,hv_arr,vec_ind_arr,act_ind_arr) = other
print('r_arr.shape=',r_arr.shape,'rp_arr.shape=',rp_arr.shape,'sample_arr.shape=',sample_arr.shape,'mask_arr.shape=',mask_arr.shape,'\n','pos_plan_arr.shape=',pos_plan_arr.shape,'pos_arr.shape=',pos_arr.shape,'dot_arr.shape=',dot_arr.shape,'policy_arr.shape=',policy_arr[0].shape,'\n','hs_arr=',hs_arr.shape,'hv_arr=',hv_arr.shape)
print('r_tot=',np.mean(np.sum(r_arr,axis=1)))
print('plan_rate=',np.sum(sample_arr,axis=None)/np.sum(mask_arr,axis=None))

r_arr.shape= (1000, 60) rp_arr.shape= (1000, 60) sample_arr.shape= (1000, 60) mask_arr.shape= (1000, 60) 
 pos_plan_arr.shape= (1000, 61, 2) pos_arr.shape= (1000, 61, 2) dot_arr.shape= (1000, 61, 2) policy_arr.shape= (1000, 60, 81) 
 hs_arr= (1000, 60, 100) hv_arr= (1000, 60, 300)
r_tot= 20.88939
plan_rate= 0.08033241


In [ ]:
sample_arr_sorted = np.argsort(np.sum(sample_arr,axis=1))[::-1]
print('sample_arr_sorted=',sample_arr_sorted[:10])

print(sample_arr[615])

In [ ]:
# binned sort
%matplotlib inline

# Calculate dot speeds
dot_diffs = np.diff(dot_arr, axis=1)
dot_speeds = np.linalg.norm(dot_diffs, axis=2)
mean_dot_speeds = np.mean(dot_speeds, axis=1)

# Define speed bins
speed_bins = np.linspace(np.min(mean_dot_speeds), np.max(mean_dot_speeds), num=5)  # Assuming 4 bins
speed_bin_centers = (speed_bins[:-1] + speed_bins[1:]) / 2

# Loop through speed bins
for i in range(len(speed_bins) - 1):
    mask = (mean_dot_speeds >= speed_bins[i]) & (mean_dot_speeds < speed_bins[i + 1])
    
    masked_indices = np.where(mask)[0]  # Keep track of original indices
    masked_decisions = sample_arr[mask]
    masked_rewards = r_arr[mask]
    
    total_rewards = np.sum(masked_rewards, axis=1)
    planning_rates = np.mean(masked_decisions, axis=1)
    product_scores = total_rewards * planning_rates

    # Get the sorted indices in descending order
    reward_indices = np.argsort(total_rewards)[::-1]
    planning_indices = np.argsort(planning_rates)[::-1]
    product_indices = np.argsort(product_scores)[::-1]

    # Top 5 indices for rewards and planning rates
    top_reward_indices = reward_indices[:5]
    top_planning_indices = planning_indices[:50]
    top_product_indices = product_indices[:10]
    
    # Map back to original indices
    original_top_reward_indices = masked_indices[top_reward_indices]
    original_top_planning_indices = masked_indices[top_planning_indices]
    original_top_product_indices = masked_indices[top_product_indices]
    
    # Print the results
    print("top rewards for speed bin", i+1)
    for idx, original_idx in enumerate(original_top_reward_indices):
        print(f" Index: {original_idx}, Reward: {total_rewards[top_reward_indices[idx]]:.2f}")

    print("\n top plan rates for speed bin", i+1)
    for idx, original_idx in enumerate(original_top_planning_indices):
        print(f"Index: {original_idx}, Planning Rate: {planning_rates[top_planning_indices[idx]]:.2f}, reward: {np.sum(r_arr[original_idx,:]):.2f}")

    print("\n top Product Scores for speed bin", i+1)
    for idx, original_idx in enumerate(original_top_product_indices):
        print(f"Index: {original_idx}, Score: {product_scores[top_product_indices[idx]]:.2f}")
    print('----------------------')

In [ ]:
# full ani pgv2/3 with policy heatmap

%matplotlib qt
import numpy as np
import scipy
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.animation as animation
import matplotlib.cm as cm
import matplotlib.colors
from matplotlib.patches import Ellipse
import functools
import jax.numpy as jnp
import jax.random as rnd

def gen_sc(keys,MODULES,ACTION_SPACE,PLAN_SPACE):
    index_range = jnp.arange(MODULES**2)
    x = jnp.linspace(-PLAN_SPACE,PLAN_SPACE,MODULES)
    y = jnp.linspace(-PLAN_SPACE,PLAN_SPACE,MODULES)[::-1]
    xv,yv = jnp.meshgrid(x,y)
    A_full = jnp.vstack([xv.flatten(),yv.flatten()])

    inner_mask = (jnp.abs(xv) <= ACTION_SPACE) & (jnp.abs(yv) <= ACTION_SPACE)
    A_inner_ind = index_range[inner_mask.flatten()]
    A_outer_ind = index_range[~inner_mask.flatten()]
    A_inner_perm = rnd.permutation(keys[0],A_inner_ind)
    A_outer_perm = rnd.permutation(keys[1],A_outer_ind)
    ID_ARR = jnp.concatenate((A_inner_perm,A_outer_perm),axis=0)

    VEC_ARR = A_full[:,ID_ARR]
    H1VEC_ARR = jnp.eye(MODULES**2) # [:,ID_ARR]
    SC = (ID_ARR,VEC_ARR,H1VEC_ARR)
    return SC #,prior_vec,zero_vec_index

def mod_(x):
    return (x + np.pi) % (2 * np.pi) - np.pi

def gen_vectors(m, A): # modules/neurons, aperture
    x = np.linspace(-(A-A/m), (A-A/m), m)
    x_ = np.tile(x, (m, ))
    y_ = np.repeat(np.flip(x), m)
    return np.vstack([x_, y_])

def circ_mean_var(v_pred,vec_range):
    v_clamped = np.clip(v_pred,0,None)
    x_y_coords = gen_vectors(np.int32(np.sqrt(len(v_clamped))),vec_range)
    x = x_y_coords[0,:]
    y = x_y_coords[1,:]
    z_x = v_pred*(np.cos(x) + 1j*np.sin(x))
    z_y = v_pred*(np.cos(y) + 1j*np.sin(y))
    mean_x = np.angle(np.sum(z_x)/np.sum(v_clamped))
    mean_y = np.angle(np.sum(z_y)/np.sum(v_clamped))
    circular_var_x = 1 - np.abs(np.sum(z_x) / np.sum(v_clamped))
    circular_var_y = 1 - np.abs(np.sum(z_y) / np.sum(v_clamped))
    circular_cov_matrix = np.diag([circular_var_x, circular_var_y])
    eigvals,eigvecs = np.linalg.eigh(circular_cov_matrix)
    sigma_x,sigma_y = np.sqrt(eigvals)  # no Scale factor for visualization
    return x_y_coords,x,y,mean_x,mean_y,sigma_x,sigma_y

def set_axis_properties(ax,lim,labels,title=''):
    ax.set_xlim(-lim,lim)
    ax.set_ylim(-lim,lim)
    ax.set_aspect('equal', 'box')  
    ax.set_title(title)
    ticks = np.linspace(-lim,lim,5)
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)
    ax.set_xticklabels(labels)
    ax.set_yticklabels(labels)

def get_aperture_points(agent_t,aperture,density):
    x_horizontal = np.linspace(agent_t[0] - aperture, agent_t[0] + aperture, density)
    y_top = np.full(density, agent_t[1] + aperture)
    y_bottom = np.full(density, agent_t[1] - aperture)
    y_vertical = np.linspace(agent_t[1] - aperture, agent_t[1] + aperture, density)
    x_left = np.full(density, agent_t[0] - aperture)
    x_right = np.full(density, agent_t[0] + aperture)
    x_tot = np.concatenate([x_left, x_right, x_horizontal, x_horizontal])
    y_tot = np.concatenate([y_vertical, y_vertical, y_bottom, y_top])
    return x_tot,y_tot

def is_point_in_ellipse(x,y,h,k,a,b):
    ellipse_eq = ((x - h)**2)/(a ** 2) + ((y - k)**2)/(b**2)
    return ellipse_eq <= 1

def vonmises_fit(x_coords,y_coords,v_pred_vec):
    scalar = 100
    kappa_max = 2
    integer_weights = np.int32(np.round(v_pred_vec*scalar))
    x_samples = np.repeat(x_coords, integer_weights)
    y_samples = np.repeat(y_coords, integer_weights)
    kappa_x,loc_x,_ = scipy.stats.vonmises.fit(x_samples)
    kappa_y,loc_y,_ = scipy.stats.vonmises.fit(y_samples)
    kappa_x = np.min([kappa_x,kappa_max])
    kappa_y = np.min([kappa_y,kappa_max])
    log_numerator = kappa_x + kappa_y
    log_denominator = 2 * np.log(2*np.pi) + np.log(scipy.special.i0(kappa_x)) + np.log(scipy.special.i0(kappa_y))
    log_result = log_numerator - log_denominator
    mean_magnitude = np.exp(log_result)
    return kappa_x,loc_x,kappa_y,loc_y,mean_magnitude

def animate(i,r_arr,dot_arr,pos_plan_arr,pos_arr,policy_arr_vec,policy_arr_act,VEC_ARR,dot,agent_plan,agent,aperture_points_plan,aperture_points,act_space_points_plan,act_space_points,policy_scatter,L):
    
    dot.set_data(mod_(dot_arr[i, 0]), mod_(dot_arr[i, 1]))
    agent_plan.set_data(mod_(pos_plan_arr[i, 0]), mod_(pos_plan_arr[i, 1]))
    agent.set_data(mod_(pos_arr[i, 0]), mod_(pos_arr[i, 1]))

    if i == 0:
        aperture_points.set_color('green')
    else:
        aperture_points.set_color('lightgrey')

    lab = f'Frame={i}, Reward={r_arr[i]:.3f}' #, k={kappa_x:.3f},{kappa_y:.3f}'
    L.get_texts()[0].set_text(lab)

    policy_dot_sizes = np.sqrt(policy_arr_vec[i,:])
    policy_color = cm.hot(colormap_norm(policy_arr_act[i,0]))

    x_aperture_plan,y_aperture_plan = get_aperture_points(pos_plan_arr[i],APERTURE,DENSITY_DEFAULT)
    aperture_points_plan.set_offsets(np.c_[mod_(x_aperture_plan), mod_(y_aperture_plan)])
    x_aperture,y_aperture = get_aperture_points(pos_arr[i],APERTURE,DENSITY_DEFAULT)
    aperture_points.set_offsets(np.c_[mod_(x_aperture), mod_(y_aperture)])
    x_act_space_plan,y_act_space_plan = get_aperture_points(pos_plan_arr[i],ACTION_SPACE,DENSITY_ACT_PLAN)
    act_space_points_plan.set_offsets(np.c_[mod_(x_act_space_plan), mod_(y_act_space_plan)])
    x_act_space,y_act_space = get_aperture_points(pos_arr[i],ACTION_SPACE,DENSITY_ACT_PLAN)
    act_space_points.set_offsets(np.c_[mod_(x_act_space), mod_(y_act_space)])

    policy_scatter.set_offsets(mod_(pos_plan_arr[i, :] + VEC_ARR.T))
    policy_scatter.set_sizes(POLICY_DOT_SIZE*policy_dot_sizes)
    policy_scatter.set_facecolors(policy_color)
    
    return [dot,agent_plan,agent,aperture_points_plan,aperture_points,act_space_points_plan,act_space_points,policy_scatter,L]

k = 493 # 68 # 902
interval = 200 # 150
APERTURE = (1/2)*np.pi # (3/5``)*np.pi # (1/2)*np.pi # (np.sqrt(2)/2)*np.pi # np.pi/2
ACTION_FRAC = 1 # 1/2
ACTION_SPACE = ACTION_FRAC*APERTURE # np.pi/4
PLAN_FRAC_REL = 1 #3/2 # 3/2
PLAN_SPACE = PLAN_FRAC_REL*ACTION_SPACE # np.pi/2
STEPS = pos_arr.shape[1]
MODULES = 9 #
SIGMA_SCALE = 3
POLICY_DOT_SIZE = 70 # 100

ke = rnd.split(rnd.PRNGKey(0),10)
(ID_ARR,VEC_ARR,H1VEC_ARR) = gen_sc(ke,MODULES,ACTION_SPACE,PLAN_SPACE)

DENSITY_DEFAULT = 1000
DENSITY_ACT_PLAN = 15 # 10

r_arr_ = r_arr[k,:]
dot_arr_ = dot_arr[k,:,:] # [STEPS,2]
pos_plan_arr_ = pos_plan_arr[k,:,:] # [STEPS,2]
pos_arr_ = pos_arr[k,:,:] # [STEPS,2]
policy_arr_vec = policy_arr[0][k,:,:]
policy_arr_act = policy_arr[1][k,:,:]

# r_arr_ = r_arr_np[k,:]
# dot_arr_ = dot_arr_np[k,:,:] # [STEPS,2]
# pos_plan_arr_ = pos_plan_arr_np[k,:,:] # [STEPS,2]
# pos_arr_ = pos_arr_np[k,:,:] # [STEPS,2]
# policy_arr_vec = policy_arr_np[0][k,:,:]
# policy_arr_act = policy_arr_np[1][k,:,:]

fig = plt.figure(figsize=(8, 12))
gs = gridspec.GridSpec(3, 2, height_ratios=[1, 1, 1])
colormap_norm = matplotlib.colors.Normalize(vmin=0, vmax=1)

ax_ani = plt.subplot(gs[0:2, 0:2],aspect='equal')
set_axis_properties(ax_ani,jnp.pi,["$-\pi$", "$-\pi/2$", "0", "$\pi/2$", "$\pi$"])
agent_plan, = ax_ani.plot(mod_(pos_plan_arr_[0, 0]), mod_(pos_plan_arr_[0, 1]), color='lightgrey', marker='+', markersize=12) #, label='agent')
agent, = ax_ani.plot(mod_(pos_arr_[0, 0]), mod_(pos_arr_[0, 1]), 'k+', markersize=12) #, label='agent')
dot, = ax_ani.plot(mod_(dot_arr_[0, 0]), mod_(dot_arr_[0, 1]), 'rx', markersize=10) #, label='dot')
text_ani, = ax_ani.plot([], [], '', label=f'Frame=0, Reward=')

aperture_points = ax_ani.scatter([], [], color='lightgrey', s=1)
aperture_points_plan = ax_ani.scatter([], [], color='lightgrey', s=1)
act_space_points = ax_ani.scatter([], [], color='lightgrey', s=1)
act_space_points_plan = ax_ani.scatter([], [], color='lightgrey', s=1)

policy_color_init = cm.hot(colormap_norm(policy_arr_act[0,0])) # Assuming policy_arr_ corresponds to policy[1]
policy_dot_sizes_init = np.sqrt(policy_arr_vec[0, :])
policy_scatter = ax_ani.scatter(VEC_ARR[0, :], VEC_ARR[1, :], s=POLICY_DOT_SIZE*policy_dot_sizes_init, alpha=1, c=policy_color_init)

L=ax_ani.legend(loc='upper right')

partial_animate = functools.partial(
    animate,
    r_arr=r_arr_,
    dot_arr=dot_arr_, 
    pos_plan_arr=pos_plan_arr_,
    pos_arr=pos_arr_, 
    policy_arr_vec=policy_arr_vec,
    policy_arr_act=policy_arr_act,
    VEC_ARR=VEC_ARR,
    dot=dot, 
    agent_plan=agent_plan,
    agent=agent, 
    aperture_points_plan=aperture_points_plan,
    aperture_points=aperture_points,
    act_space_points_plan=act_space_points_plan,
    act_space_points=act_space_points,
    policy_scatter=policy_scatter,
    L=L,
    )

ani = animation.FuncAnimation(
    fig=fig, 
    func=partial_animate,
    frames=STEPS, 
    blit=True, 
    interval=interval
    )

plt.tight_layout()
plt.show()

In [171]:
# NEURON-FOCUSED (RIGHT SINGULAR VECTORS)
# PCA using SVD; get low-dim trajectories of top K PC's

import numpy as np

K = 5 # 10

def pca_svd_method(data, K):
    # Reshape the data
    num_samples, T, H = data.shape
    reshaped_data = data.reshape(num_samples*T, H)
    
    # Center the data
    mean_data = np.mean(reshaped_data, axis=0)
    centered_data = reshaped_data - mean_data
    
    # Compute the SVD
    U, S, V = np.linalg.svd(centered_data, full_matrices=False)
    
    # Get the top K columns of V
    V_topK = V.T[:, :K]
    
    # Project data onto the top K PCs
    projected_data = centered_data @ V_topK
    
    # Reshape the projected data back to [1000, 60, K]
    projected_data = projected_data.reshape(num_samples, T, K)
    
    return projected_data

hs_pc_s = pca_svd_method(hs_arr, K)
hv_pc_s = pca_svd_method(hv_arr, K)

print(hs_pc_s.shape)  # Should print (1000, 60, 3)
print(hv_pc_s.shape)  # Should print (1000, 60, 3)

(1000, 60, 5)
(1000, 60, 5)


In [ ]:
# NEURON-FOCUSED
# PCA using covar matrix; get low-dim trajectories of top K PC's

import numpy as np

K = 3

def pca_covariance_method(data, K):
    # Reshape the data to stack all time points across all trials
    num_samples, T, H = data.shape
    reshaped_data = data.reshape(num_samples*T, H)

    # Subtract the overall mean across trials and time points
    mean_data = np.mean(reshaped_data, axis=0)
    centered_data = reshaped_data - mean_data
    
    # Calculate covariance matrix
    cov_matrix = np.cov(centered_data, rowvar=False)
    
    # Eigen decomposition
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
    
    # Sorting eigenvectors based on eigenvalues
    idx = eigenvalues.argsort()[::-1]  # Sort in descending order
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]

    # Take the top K eigenvectors
    top_eigenvectors = eigenvectors[:, :K]
    
    # Project data onto top K eigenvectors to get principal components
    projected_data = np.array([np.dot(data[i], top_eigenvectors) for i in range(data.shape[0])])
    
    return projected_data

hs_pc_c = pca_covariance_method(hs_arr, K)
hv_pc_c = pca_covariance_method(hv_arr, K)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

%matplotlib inline

# Assuming 'full_activity' is your original data array of shape (1000, 60, 100)
# Reshape the data for PCA: combining trials and timesteps, resulting in a (1000*60, 100) array
full_activity_reshaped = hs_arr.reshape(-1, hs_arr.shape[2])

# Perform PCA
pca = PCA(n_components=20)  # Adjust the number of components as needed
pca.fit(full_activity_reshaped)

# Get the explained variance ratio of each component
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

# Plotting
plt.figure(figsize=(10, 6))
plt.bar(range(len(explained_variance)), explained_variance, alpha=0.5, align='center', label='Individual explained variance')
plt.step(range(len(cumulative_variance)), cumulative_variance, where='mid', label='Cumulative explained variance')
plt.ylabel('Explained Variance Ratio')
plt.xlabel('Principal Components')
plt.xticks(range(len(explained_variance)))
plt.legend(loc='best')
plt.title('Variance Explained by Principal Components')
plt.show()

In [ ]:
task_param=np.linalg.norm(np.mean(np.diff(pos_arr,axis=1),axis=1),axis=1)
print('task_param=',task_param.shape,task_param[0:10])

In [ ]:
# speed decoding performance (single split)

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Assuming pca_data is your PCA-transformed data of shape (1000, 60, number_of_PCs)
# Assuming task_param is your task parameter vector of length 1000

max_timesteps = 60  # Maximum number of timesteps
pcs_scenarios = [1, 3, 5]  # Top 1, top 5, and top 10 PCs
rmses = {scenario: [] for scenario in pcs_scenarios}

# Calculate the mean baseline RMSE
mean_task_param = np.mean(task_param)
baseline_rmse = np.sqrt(mean_squared_error(task_param, np.full_like(task_param, mean_task_param)))

for T in range(1, max_timesteps + 1):
    for pcs in pcs_scenarios:
        # Select the first T timesteps and first 'pcs' PCs
        X_flattened = hs_pc_s[:, :T, :pcs].reshape(-1, T * pcs)
        
        # Split the data
        X_train, X_test, y_train, y_test = train_test_split(X_flattened, task_param, test_size=0.2, random_state=42)
        
        # Train the model
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Predict on the test set and calculate RMSE
        y_pred = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        rmses[pcs].append(rmse)

# Plotting
plt.figure(figsize=(10, 6))
plt.axhline(y=baseline_rmse, color='r', linestyle='-', label='Mean Baseline')
for pcs, rmse_values in rmses.items():
    plt.plot(range(1, max_timesteps + 1), rmse_values, marker='o', label=f'Top {pcs} PCs')

plt.xlabel('Number of Timesteps (T)')
plt.ylabel('RMSE')
plt.title('Decoding Performance with Varying Timesteps and PCs')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# no pcs

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from scipy.stats import sem

# Assuming pca_data is your PCA-transformed data of shape (1000, 60, 20)
# And task_param is your task parameter vector of length 1000

pca_data = hs_pc_s
max_pcs = 10  # Maximum number of PCs
rmses = []
sem_values = []  # To store SEM values for the model
random_rmses = []  # To store RMSEs for random baseline
n_splits = 5  # Number of folds for K-Fold Cross-Validation

task_param = np.linalg.norm(np.mean(np.diff(pos_arr, axis=1), axis=1), axis=1)
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Function to generate random predictions
def generate_random_predictions(size):
    return np.random.choice(task_param, size=size)

# Calculate RMSE and SEM for random baseline across folds
for train_index, test_index in kf.split(task_param):
    y_test = task_param[test_index]
    random_preds = generate_random_predictions(size=len(test_index))
    random_rmses.append(np.sqrt(mean_squared_error(y_test, random_preds)))

baseline_rmse = np.mean(random_rmses)
baseline_sem = sem(random_rmses)

# Model performance calculation
for k in range(1, max_pcs + 1):
    fold_rmses = []
    X_flattened = pca_data[:, :, :k].reshape(-1, 60*k)

    for train_index, test_index in kf.split(X_flattened):
        X_train, X_test = X_flattened[train_index], X_flattened[test_index]
        y_train, y_test = task_param[train_index], task_param[test_index]

        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        fold_rmses.append(np.sqrt(mean_squared_error(y_test, y_pred)))

    rmses.append(np.mean(fold_rmses))
    sem_values.append(sem(fold_rmses))

# Plotting
plt.figure(figsize=(10, 6))
plt.axhline(y=baseline_rmse, color='red', linestyle='-', label='Random Baseline')
plt.fill_between(range(1, max_pcs + 1), [baseline_rmse - baseline_sem]*max_pcs, 
                 [baseline_rmse + baseline_sem]*max_pcs, color='lightgrey', alpha=0.5, label='Baseline SEM')
for k in range(1, max_pcs + 1):
    plt.plot(range(1, max_pcs + 1), rmses, marker='o', label='Model RMSE')
    plt.fill_between(range(1, max_pcs + 1), np.array(rmses) - np.array(sem_values), 
                     np.array(rmses) + np.array(sem_values), color='lightgrey', alpha=0.5, label='Model SEM')

plt.xlabel('Number of Principal Components Used')
plt.ylabel('RMSE')
plt.title('RMSE vs. Number of Principal Components')
# plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# no pcs multi
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from scipy.stats import sem

# Assuming pca_data is your PCA-transformed data of shape (1000, 60, 20)
# Assuming pos_arr is defined and used to calculate task_param

# Calculate task_param assuming pos_arr is correctly defined
task_param = np.mean(np.diff(pos_arr, axis=1), axis=1)  # Check if this is the correct way to calculate task_param

pca_data = hs_pc_s
max_pcs = 10  # Maximum number of PCs
n_splits = 5  # Number of folds for K-Fold Cross-Validation

kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Function to generate random predictions for multi-output regression
def generate_random_predictions(size, num_outputs):
    return np.random.uniform(low=-1.0, high=1.0, size=(size, num_outputs))  # Adjust range as per your data

# Initialize lists to store RMSEs and SEMs for each output
num_outputs = task_param.shape[1]
rmses = np.zeros((max_pcs, num_outputs))
sem_values = np.zeros((max_pcs, num_outputs))
random_rmses = []

# Calculate RMSE for random baseline across folds
for train_index, test_index in kf.split(pca_data):
    y_test = task_param[test_index]
    random_preds = generate_random_predictions(len(test_index), num_outputs)
    random_rmses.append(np.sqrt(mean_squared_error(y_test, random_preds, multioutput='raw_values')))

baseline_rmse = np.mean(random_rmses, axis=0)
baseline_sem = sem(random_rmses, axis=0)

# Model performance calculation for each number of PCs
for k in range(max_pcs):
    fold_rmses = []
    X_flattened = pca_data[:, :, :k+1].reshape(-1, 60 * (k+1))

    for train_index, test_index in kf.split(X_flattened):
        X_train, X_test = X_flattened[train_index], X_flattened[test_index]
        y_train, y_test = task_param[train_index], task_param[test_index]

        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        fold_rmses.append(np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values')))

    rmses[k, :] = np.mean(fold_rmses, axis=0)
    sem_values[k, :] = sem(fold_rmses, axis=0)

# Plotting
plt.figure(figsize=(10, 6))
for i in range(num_outputs):
    plt.plot(range(1, max_pcs + 1), rmses[:, i], marker='o', label=f'Output {i+1} RMSE')
    plt.fill_between(range(1, max_pcs + 1), rmses[:, i] - sem_values[:, i], rmses[:, i] + sem_values[:, i], alpha=0.5)

# Baseline plots
for i in range(num_outputs):
    plt.axhline(y=baseline_rmse[i], linestyle='-', label=f'Random Baseline {i+1}') 
    plt.fill_between(range(1, max_pcs + 1), [baseline_rmse[i] - baseline_sem[i]] * max_pcs, 
                     [baseline_rmse[i] + baseline_sem[i]] * max_pcs, alpha=0.2)

plt.xlabel('Number of Principal Components Used')
plt.ylabel('RMSE')
plt.title('RMSE vs. Number of Principal Components for Each Output')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# # speed k fold

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.stats import sem

# Assuming pca_data is your PCA-transformed data of shape (1000, 60, number_of_PCs)
# Assuming task_param is your task parameter vector of length 1000

max_timesteps = 60  # Maximum number of timesteps
pcs_scenarios = [1, 3, 5]  # Top 1, top 5, and top 10 PCs
rmses = {scenario: [] for scenario in pcs_scenarios}
sems = {scenario: [] for scenario in pcs_scenarios}
n_splits = 5  # Number of folds in cross-validation
kf = KFold(n_splits=n_splits)

clrs=['#ADD8E6','#1E90FF','#00008B']

# Function to generate random predictions
def generate_random_predictions(y, num_samples):
    return np.random.choice(y, size=num_samples)

# Calculate RMSE and SEM for random baseline
random_rmse = []
for _ in range(1000):  # Bootstrap iterations
    random_preds = generate_random_predictions(task_param, len(task_param))
    random_rmse.append(np.sqrt(mean_squared_error(task_param, random_preds)))
baseline_rmse = np.mean(random_rmse)
baseline_sem = sem(random_rmse)

for T in range(1, max_timesteps + 1):
    for pcs in pcs_scenarios:
        rmse_fold = []
        X_flattened = hs_pc_s[:, :T, :pcs].reshape(-1, T * pcs)

        for train_index, test_index in kf.split(X_flattened):
            X_train, X_test = X_flattened[train_index], X_flattened[test_index]
            y_train, y_test = task_param[train_index], task_param[test_index]

            model = LinearRegression()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            rmse_fold.append(np.sqrt(mean_squared_error(y_test, y_pred)))

        rmses[pcs].append(np.mean(rmse_fold))
        sems[pcs].append(sem(rmse_fold))

# Plotting
plw=6
plt.figure(figsize=(7, 6))
plt.axhline(y=baseline_rmse, color='r', linestyle='-', label='random')
plt.fill_between(range(1, max_timesteps + 1), [baseline_rmse - baseline_sem] * max_timesteps, 
                 [baseline_rmse + baseline_sem] * max_timesteps, color='red', alpha=0.2)  # Baseline SEM shading

for (pcs, rmse_values), color in zip(rmses.items(), clrs):
    plt.plot(range(1, max_timesteps + 1), rmse_values, label=f'{pcs} PCs', color=color, linewidth=plw)
    plt.fill_between(range(1, max_timesteps + 1), 
                     np.array(rmse_values) - np.array(sems[pcs]), 
                     np.array(rmse_values) + np.array(sems[pcs]), 
                     alpha=0.5, color='lightgrey')  # Model SEM shading
    
lw = 4
fnt = 42
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_linewidth(lw)
ax.spines['bottom'].set_linewidth(lw)
ax.tick_params(axis='both', which='major', labelsize=fnt, length=10, width=lw)  # Adjust length and width as needed
ax.set_xticks([0, 30, 60])
ax.set_yticks([0.05, 0.09, 0.13])


plt.xlabel('$\#$ timesteps', fontsize=fnt, labelpad=0)
plt.ylabel('RMSE', fontsize=fnt, labelpad=0)
# plt.title('Decoding Performance with Varying Timesteps and PCs')
plt.legend(loc='center right', ncols =1, fontsize=30, frameon=False, bbox_to_anchor=(1.0, 0.5))
# plt.grid(True)
plt.tight_layout()
plt.show()

print('baseline_sem=',baseline_sem)

In [ ]:
# multi-output k fold

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Assuming pca_data is your PCA-transformed data of shape (1000, 60, number_of_PCs)
# Assuming task_param is your task parameter matrix of shape (1000, 2)

clrs=['#ADD8E6','#1E90FF','#00008B']

# Function to generate random predictions for multi-output regression
def generate_random_predictions(num_samples, num_targets, target_ranges):
    # 'target_ranges' should be a list of arrays, where each array is the range of possible values for each target
    return np.array([np.random.choice(target_ranges[i], size=num_samples) for i in range(num_targets)]).T

task_param=np.mean(np.diff(pos_arr,axis=1),axis=1)

# Calculate RMSE for random baseline
baseline_rmses = []
target_ranges = [np.unique(task_param[:, i]) for i in range(task_param.shape[1])]  # Assuming task_param is 2D

for T in range(1, max_timesteps + 1):
    random_preds = generate_random_predictions(len(task_param), task_param.shape[1], target_ranges=target_ranges)
    baseline_rmse_output1 = np.sqrt(mean_squared_error(task_param[:, 0], random_preds[:, 0]))
    baseline_rmse_output2 = np.sqrt(mean_squared_error(task_param[:, 1], random_preds[:, 1]))
    baseline_rmses.append((baseline_rmse_output1 + baseline_rmse_output2) / 2)

max_timesteps = 60  # Maximum number of timesteps
pcs_scenarios = [1, 3, 5]  # Top 1, top 5, and top 10 PCs
rmses = {scenario: [] for scenario in pcs_scenarios}

n_splits = 5  # Number of folds in cross-validation
kf = KFold(n_splits=n_splits)

for T in range(1, max_timesteps + 1):
    for pcs in pcs_scenarios:
        rmse_fold = []
        X_flattened = hs_pc_s[:, :T, :pcs].reshape(-1, T * pcs)

        for train_index, test_index in kf.split(X_flattened):
            X_train, X_test = X_flattened[train_index], X_flattened[test_index]
            y_train, y_test = task_param[train_index], task_param[test_index]

            model = LinearRegression()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Calculate RMSE for each output and average
            rmse_output1 = np.sqrt(mean_squared_error(y_test[:, 0], y_pred[:, 0]))
            rmse_output2 = np.sqrt(mean_squared_error(y_test[:, 1], y_pred[:, 1]))
            rmse_fold.append((rmse_output1 + rmse_output2) / 2)

        rmses[pcs].append(np.mean(rmse_fold))

# Plotting
plt.figure(figsize=(10, 8))  # Increased figure size for clarity
for pcs, rmse_values in rmses.items():
    plt.plot(range(1, max_timesteps + 1), rmse_values, marker='o', label=f'Top {pcs} PCs', linewidth=2)

# Plot baseline RMSE
plt.plot(range(1, max_timesteps + 1), baseline_rmses, color='red', linestyle='--', label='Random Baseline', linewidth=2)

plt.xlabel('Number of Timesteps (T)', fontsize=14)
plt.ylabel('Average RMSE', fontsize=14)
plt.title('Multi-output Decoding Performance', fontsize=16)
plt.legend()
plt.grid(True)
plt.tight_layout()

plt.show()


In [225]:
# # multi output k fold sem
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.stats import sem

# clrs=['#ADD8E6','#1E90FF','#00008B']

# def generate_random_predictions(num_samples, num_targets, target_ranges):
#     return np.array([np.random.choice(target_ranges[i], size=num_samples) for i in range(num_targets)]).T

# task_param=np.mean(np.diff(pos_arr,axis=1),axis=1)

# baseline_rmses_output1 = []
# baseline_rmses_output2 = []
# target_ranges = [np.unique(task_param[:, i]) for i in range(task_param.shape[1])]  # Assuming task_param is 2D

# for T in range(1, max_timesteps + 1):
#     random_preds = generate_random_predictions(len(task_param), task_param.shape[1], target_ranges=target_ranges)
#     baseline_rmse_output1 = np.sqrt(mean_squared_error(task_param[:, 0], random_preds[:, 0]))
#     baseline_rmse_output2 = np.sqrt(mean_squared_error(task_param[:, 1], random_preds[:, 1]))
#     baseline_rmses_output1.append(baseline_rmse_output1)
#     baseline_rmses_output2.append(baseline_rmse_output2)

# # Calculate combined baseline RMSE
# combined_baseline_rmse = np.mean([np.mean(baseline_rmses_output1), np.mean(baseline_rmses_output2)])

# # Calculate SEM for each output
# sem_output1 = np.std(baseline_rmses_output1, ddof=1) / np.sqrt(len(baseline_rmses_output1))
# sem_output2 = np.std(baseline_rmses_output2, ddof=1) / np.sqrt(len(baseline_rmses_output2))

# # Optionally, calculate the combined SEM
# combined_baseline_sem = np.mean([sem_output1, sem_output2])

# max_timesteps = 60  # Maximum number of timesteps
# pcs_scenarios = [1, 3, 5]  # Top 1, top 3, and top 5 PCs
# rmses = {scenario: [] for scenario in pcs_scenarios}
# sems = {scenario: [] for scenario in pcs_scenarios}  # Store SEMs here

# n_splits = 5  # Number of folds in cross-validation
# kf = KFold(n_splits=n_splits)

# for T in range(1, max_timesteps + 1):
#     for pcs in pcs_scenarios:
#         rmse_fold = []
#         X_flattened = hs_pc_s[:, :T, :pcs].reshape(-1, T * pcs)

#         for train_index, test_index in kf.split(X_flattened):
#             X_train, X_test = X_flattened[train_index], X_flattened[test_index]
#             y_train, y_test = task_param[train_index], task_param[test_index]

#             model = LinearRegression()
#             model.fit(X_train, y_train)
#             y_pred = model.predict(X_test)

#             rmse_output1 = np.sqrt(mean_squared_error(y_test[:, 0], y_pred[:, 0]))
#             rmse_output2 = np.sqrt(mean_squared_error(y_test[:, 1], y_pred[:, 1]))
#             rmse_fold.append((rmse_output1 + rmse_output2) / 2)

#         rmses[pcs].append(np.mean(rmse_fold))
#         sems[pcs].append(sem(rmse_fold))  # Calculate and append SEM

# Plotting
plw = 6
plt.figure(figsize=(6.5, 6))

# Plot a single line for the combined average random baseline RMSE and SEM
plt.axhline(y=combined_baseline_rmse, color='r', linestyle='--', linewidth=6,label='random')
plt.fill_between(range(1, max_timesteps + 1), 
                 [combined_baseline_rmse - combined_baseline_sem] * max_timesteps, 
                 [combined_baseline_rmse + combined_baseline_sem] * max_timesteps, 
                 color='lightgrey', alpha=0.2)

# Plot RMSE and SEM for each number of PCs
for (pcs, rmse_values), color in zip(rmses.items(), clrs):
    if pcs == 1:
        label_text = f'{pcs} PC'  # Singular
    else:
        label_text = f'{pcs} PCs'  # Plural

    plt.plot(range(1, max_timesteps + 1), rmse_values, label=label_text, color=color, linewidth=plw)
    plt.fill_between(range(1, max_timesteps + 1), 
                     np.array(rmse_values) - np.array(sems[pcs]), 
                     np.array(rmse_values) + np.array(sems[pcs]), 
                     alpha=0.5, color='lightgrey')

# Styling axes and labels
lw = 4
fnt = 38
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_linewidth(lw)
ax.spines['bottom'].set_linewidth(lw)
ax.tick_params(axis='both', which='major', labelsize=fnt, length=10, width=lw)
ax.set_xticks([0, 30, 60])
ax.set_yticks([0,0.1,0.2])
ax.set_ylim([0,0.24])

plt.xlabel('$\#$ timesteps', fontsize=fnt)
plt.ylabel('RMSE', fontsize=fnt)
plt.legend(loc='upper center', ncols=2, fontsize=28, frameon=False, handlelength=1.3, handletextpad=0.5, columnspacing=0.4, bbox_to_anchor=(0.53, 1.27))
plt.tight_layout()
plt.subplots_adjust(top=0.85)  # Adjust this value as needed
plt.show()

2023-12-14 00:19:12.211 Python[96371:10875106] +[CATransaction synchronize] called within transaction


In [ ]:
# multi output k fold sem (broken)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.stats import sem

clrs=['#ADD8E6','#1E90FF','#00008B']

def generate_random_predictions(num_samples, num_targets, target_ranges):
    return np.array([np.random.choice(target_ranges[i], size=num_samples) for i in range(num_targets)]).T

task_param=np.mean(np.diff(pos_arr,axis=1),axis=1)

target_ranges = [np.unique(task_param[:, i]) for i in range(task_param.shape[1])]  # Assuming task_param is 2D
for T in range(1, max_timesteps + 1):
    random_preds = generate_random_predictions(len(task_param), task_param.shape[1], target_ranges=target_ranges)
    baseline_rmse_output1 = np.sqrt(mean_squared_error(task_param[:, 0], random_preds[:, 0]))
    baseline_rmse_output2 = np.sqrt(mean_squared_error(task_param[:, 1], random_preds[:, 1]))
    baseline_rmses.append((baseline_rmse_output1 + baseline_rmse_output2) / 2)

baseline_rmse = np.mean(random_preds)
baseline_sem = sem(baseline_rmses)

combined_baseline_rmse = np.mean(baseline_rmse)
combined_baseline_sem = np.mean(baseline_sem)

max_timesteps = 60  # Maximum number of timesteps
pcs_scenarios = [1, 3, 5]  # Top 1, top 3, and top 5 PCs
rmses = {scenario: [] for scenario in pcs_scenarios}
sems = {scenario: [] for scenario in pcs_scenarios}  # Store SEMs here

n_splits = 5  # Number of folds in cross-validation
kf = KFold(n_splits=n_splits)

for T in range(1, max_timesteps + 1):
    for pcs in pcs_scenarios:
        rmse_fold = []
        X_flattened = hs_pc_s[:, :T, :pcs].reshape(-1, T * pcs)

        for train_index, test_index in kf.split(X_flattened):
            X_train, X_test = X_flattened[train_index], X_flattened[test_index]
            y_train, y_test = task_param[train_index], task_param[test_index]

            model = LinearRegression()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            rmse_output1 = np.sqrt(mean_squared_error(y_test[:, 0], y_pred[:, 0]))
            rmse_output2 = np.sqrt(mean_squared_error(y_test[:, 1], y_pred[:, 1]))
            rmse_fold.append((rmse_output1 + rmse_output2) / 2)

        rmses[pcs].append(np.mean(rmse_fold))
        sems[pcs].append(sem(rmse_fold))  # Calculate and append SEM

# Plotting
plw = 6
plt.figure(figsize=(7, 6))

# Plot a single line for the combined average random baseline RMSE and SEM
plt.axhline(y=combined_baseline_rmse, color='r', linestyle='--', linewidth=4,label='random')
plt.fill_between(range(1, max_timesteps + 1), 
                 [combined_baseline_rmse - combined_baseline_sem] * max_timesteps, 
                 [combined_baseline_rmse + combined_baseline_sem] * max_timesteps, 
                 color='lightgrey', alpha=0.2)

# Plot RMSE and SEM for each number of PCs
for (pcs, rmse_values), color in zip(rmses.items(), clrs):
    if pcs == 1:
        label_text = f'{pcs} PC'  # Singular
    else:
        label_text = f'{pcs} PCs'  # Plural

    plt.plot(range(1, max_timesteps + 1), rmse_values, label=label_text, color=color, linewidth=plw)
    plt.fill_between(range(1, max_timesteps + 1), 
                     np.array(rmse_values) - np.array(sems[pcs]), 
                     np.array(rmse_values) + np.array(sems[pcs]), 
                     alpha=0.5, color='lightgrey')

# Styling axes and labels
lw = 4
fnt = 38
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_linewidth(lw)
ax.spines['bottom'].set_linewidth(lw)
ax.tick_params(axis='both', which='major', labelsize=fnt, length=10, width=lw)
ax.set_xticks([0, 30, 60])
ax.set_yticks([0.6, 0.4, 0.2,0])

plt.xlabel('$\#$ timesteps', fontsize=fnt)
plt.ylabel('RMSE', fontsize=fnt)
plt.legend(loc='center right', ncols=1, fontsize=30, frameon=False, bbox_to_anchor=(1.0, 0.55), handlelength=1.5)
plt.tight_layout()
plt.show()

In [ ]:
# plot specified trajectories; plans in black

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d.art3d import Line3DCollection

def plot_trajectories(fig, data, sample_arr, n_list):
    colormaps = ['PiYG', 'PRGn', 'PuOr', 'RdBu']
    
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122)
    
    for idx, n in enumerate(n_list):
        traj = data[n, :, :3]
        length = traj.shape[0]
        
        # Compute transitions in sample_arr for current trajectory
        transitions = np.where(np.diff(sample_arr[n, :]) != 0)[0]
        print('tr=',transitions)
        
        # Add start or end point if the trajectory starts or ends with '1'
        if sample_arr[n, 0] == 1:
            transitions = np.insert(transitions, 0, -1)
        if sample_arr[n, -1] == 1:
            transitions = np.append(transitions, length-2)
        
        # Create color array based on sample_arr
        colors = np.array([plt.get_cmap(colormaps[idx])(i) if sample_arr[n, i] == 0 else (0, 0, 0, 1) for i in range(length)])
        
        segments_3d = [traj[i:i+2] for i in range(length-1)]
        segments_2d = [s[:, :2] for s in segments_3d]
        
        lc_3d = Line3DCollection(segments_3d, colors=colors[:-1])
        lc_2d = LineCollection(segments_2d, colors=colors[:-1])
        
        ax1.add_collection(lc_3d)
        ax2.add_collection(lc_2d)

        # Add dots for transitions
        if sample_arr[n, 0] == 1:
            ax1.scatter(*traj[0, :3], color='black', s=10)
            ax2.scatter(*traj[0, :2], color='black', s=10)

        if sample_arr[n, -1] == 1:
            ax1.scatter(*traj[-1, :3], color='black', s=10)
            ax2.scatter(*traj[-1, :2], color='black', s=10)

        for t in transitions:
            ax1.scatter(*traj[t+1, :3], color='black', s=10)
            ax2.scatter(*traj[t+1, :2], color='black', s=10)

    # Set limits based on data
    all_data = data[:, :, :3].reshape(-1, 3)
    ax1.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax1.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    ax1.set_zlim(all_data[:, 2].min(), all_data[:, 2].max())
    ax2.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax2.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())

    ax1.set_title('3D Trajectories')
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    ax1.set_zlabel('PC3')
    ax2.set_title('2D Projections of Trajectories')
    ax2.set_xlabel('PC1')
    ax2.set_ylabel('PC2')
    
    plt.show()

fig = plt.figure(figsize=(12, 6))
n_list = [615, 616] # 133
# [884, 87, 20] # R2
# [335,796,479] # R1
plot_trajectories(fig, hs_pc_s, sample_arr, n_list)

In [ ]:
### colorwheel (correct)
# top T trajectories binned by dot vector (avg/all)

%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.colors import LinearSegmentedColormap

def generate_bin_vectors(N):
    angles = np.linspace(0, 2 * np.pi, N+1)[:-1]  # Omitting the last angle to avoid overlap with 0
    return np.column_stack((np.cos(angles), np.sin(angles)))

# def has_duplicates(array_2d):
#     flat_list = array_2d.flatten().tolist()
#     unique_set = set(flat_list)
#     return len(unique_set) < len(flat_list)

def generate_polar_angles(N):
    step = 360 / N
    angles = [(i * step) % 360 for i in range(N)]
    return [f"{int(angle)}°" if angle <= 180 else f"{int(angle - 360)}°" for angle in angles]

def compute_dot_direction(dot_arr):
    dot_diffs = np.diff(dot_arr, axis=1)
    mean_dot_dir = np.mean(dot_diffs, axis=1)
    norm_dot_dir = np.linalg.norm(mean_dot_dir, axis=1)
    mean_dot_dir_normalized = mean_dot_dir / norm_dot_dir[:, np.newaxis]
    return mean_dot_dir_normalized

def get_top_rewarded_trials_new(dot_arr, r_arr, T, N):
    bins = generate_bin_vectors(N)
    normalized_dot_dirs = compute_dot_direction(dot_arr)

    # Assign each dot to a bin
    bin_assignments = []
    for dot in normalized_dot_dirs:
        cosine_similarity = dot @ np.array(bins).T
        assigned_bin = np.argmax(cosine_similarity)
        bin_assignments.append(assigned_bin)

    # Organize trials by their assigned bin and then sort them by total reward
    total_rewards = np.sum(r_arr, axis=1)
    bin_lists = [[] for _ in range(N)]
    for i, assigned_bin in enumerate(bin_assignments):
        bin_lists[assigned_bin].append(i)
    
    # Sort trials in each bin by reward and select top T trials
    top_trials = np.zeros((T, N), dtype=int)
    for idx, trial_list in enumerate(bin_lists):
        sorted_trials = sorted(trial_list, key=lambda x: total_rewards[x], reverse=True)
        top_trials[:, idx] = sorted_trials[:T]

    return top_trials

def plot_trajectories(fig, data, top_trials, N):
    # Define a color wheel
    color_wheel = plt.cm.hsv(np.linspace(0, 1, N+1))
    custom_cmap = LinearSegmentedColormap.from_list('custom_cmap', color_wheel)
    
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122)
    
    for bin_idx, trials in enumerate(top_trials.T): # Transpose to iterate over trials per bin
        for trial_idx in trials:
            traj = data[trial_idx, :, :3]
            segments_3d = [traj[i:i+2] for i in range(traj.shape[0]-1)]
            segments_2d = [s[:, :2] for s in segments_3d]
            
            lc_3d = Line3DCollection(segments_3d, colors=color_wheel[bin_idx])
            lc_2d = LineCollection(segments_2d, colors=color_wheel[bin_idx])
            
            ax1.add_collection(lc_3d)
            ax2.add_collection(lc_2d)
    
    # Set limits based on data
    all_data = data[:, :, :3].reshape(-1, 3)
    ax1.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax1.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    ax1.set_zlim(all_data[:, 2].min(), all_data[:, 2].max())
    ax2.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax2.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())

    polar_angles = generate_polar_angles(N) # ["0°", "45°", "90°", "135°", "±180°", "-135°", "-90°", "-45°"]
    legend_elements = [Line2D([0], [0], color=color_wheel[i], lw=2, label=polar_angles[i]) for i in range(N)]
    ax2.legend(handles=legend_elements, loc="upper right")
    
    ax1.set_title('3D Trajectories')
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    ax1.set_zlabel('PC3')
    ax2.set_title('2D Projections of Trajectories')
    ax2.set_xlabel('PC1')
    ax2.set_ylabel('PC2')
    
    plt.show()

def plot_avg_trajectories(fig, data, top_trials, N):
    # Define a color wheel
    color_wheel = plt.cm.hsv(np.linspace(0, 1, N+1))
    custom_cmap = LinearSegmentedColormap.from_list('custom_cmap', color_wheel)
    
    ax1 = fig.add_subplot(121, projection='3d')
    # ax2 = fig.add_subplot(122)
    
    for bin_idx, trials in enumerate(top_trials.T): 
        # Calculate the average trajectory for the trials in this bin
        avg_traj = np.mean(data[trials], axis=0)
        
        segments_3d = [avg_traj[i:i+2] for i in range(avg_traj.shape[0]-1)]
        segments_2d = [s[:, :2] for s in segments_3d]
        
        lc_3d = Line3DCollection(segments_3d, colors=color_wheel[bin_idx])
        lc_2d = LineCollection(segments_2d, colors=color_wheel[bin_idx])
        
        ax1.add_collection(lc_3d)
        # ax2.add_collection(lc_2d)
    
    top_data_indices = top_trials.flatten()  # Assuming top_trials is a 2D array of indices
    top_data = data[top_data_indices, :, :3].reshape(-1, 3)

    # Define padding as a percentage of the data range
    padding_factor = -0.0  # 5% padding

    # Calculate the limits based on the top_data with padding
    x_padding = (top_data[:, 0].max() - top_data[:, 0].min()) * padding_factor
    y_padding = (top_data[:, 1].max() - top_data[:, 1].min()) * padding_factor
    z_padding = (top_data[:, 2].max() - top_data[:, 2].min()) * padding_factor

    x_range = (top_data[:, 0].min() - x_padding, top_data[:, 0].max() + x_padding)
    y_range = (top_data[:, 1].min() - y_padding, top_data[:, 1].max() + y_padding)
    z_range = (top_data[:, 2].min() - z_padding, top_data[:, 2].max() + z_padding)

    # Set the limits for each axis based on the top_data with padding
    ax1.set_xlim(x_range)
    ax1.set_ylim(y_range)
    ax1.set_zlim(z_range)

    sm = plt.cm.ScalarMappable(cmap=custom_cmap, norm=plt.Normalize(vmin=0, vmax=N))
    sm.set_array([])  # You can set an array of values here if needed for scaling
    cbar = plt.colorbar(sm, orientation='horizontal', ticks=[], aspect=4)
    cbar.outline.set_edgecolor('none')  # Remove the colorbar border

    # Manually adjust the size and position of the colorbar
    cbar_width = ax1.get_position().width
    cbar_height = 0.025  # Set the height of the colorbar
    cbar_pos_x = ax1.get_position().x0
    cbar_pos_y = ax1.get_position().y1 # + 0.05  # Adjust vertical position
    cbar.ax.set_position([cbar_pos_x - 0.03, cbar_pos_y - .07, cbar_width, cbar_height])

    # Remove tick marks from the colorbar
    cbar.ax.xaxis.set_ticks_position('top')
    cbar.ax.xaxis.set_tick_params(size=0)

    # Add the 'time' label to the right
    cbar.ax.text(1.2, 0.5, 'angle', verticalalignment='center', transform=cbar.ax.transAxes, fontsize=12)

    # ax1.set_box_aspect([1,1,1]) 
    # ax2.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    # ax2.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    
    # Create a legend with polar angles corresponding to the bins
    polar_angles = generate_polar_angles(N) # ["0°", "45°", "90°", "135°", "±180°", "-135°", "-90°", "-45°"]
    legend_elements = [Line2D([0], [0], color=color_wheel[i], lw=2, label=polar_angles[i]) for i in range(N)]
    # ax2.legend(handles=legend_elements, loc="upper right")
    
    # x_min, x_max = all_data[:, 0].min(), all_data[:, 0].max()
    # y_min, y_max = all_data[:, 1].min(), all_data[:, 1].max()
    # z_min, z_max = all_data[:, 2].min(), all_data[:, 2].max()

    # ax1.set_title('3D Average Trajectories')
    ax1.set_xlabel('PC1', fontsize=fnt, labelpad=-5, rotation=0)
    ax1.set_xticks([])
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['left'].set_linewidth(4)
    ax1.spines['bottom'].set_linewidth(4)
    # ax1.xaxis.pane.fill = False
    ax1.set_ylabel('PC2', fontsize=fnt, labelpad=-5, rotation=0)
    ax1.set_yticks([])
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['left'].set_linewidth(4)
    ax1.spines['bottom'].set_linewidth(4)
    ax1.set_zlabel('PC3', fontsize=fnt, labelpad=-10, rotation=90)
    ax1.set_zticks([])
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['left'].set_linewidth(4)
    ax1.spines['bottom'].set_linewidth(4)
    # ax2.set_title('2D Projections of Average Trajectories')
    # ax2.set_xlabel('PC1')
    # ax2.set_ylabel('PC2')
    ax1.grid(False)
    # ax1.legend(handles=legend_elements, loc="upper right", fontsize=fnt, frameon=False, bbox_to_anchor=(1.0, 0.5))
    
    plt.show()

N = 8 # 8
T = 50
fnt = 15

# Assuming hs_pc_s data is available
fig = plt.figure(figsize=(8, 6))
top_rewarded_trials = get_top_rewarded_trials_new(dot_arr, r_arr, T, N)
print('trt=',top_rewarded_trials.shape)
plot_avg_trajectories(fig, hs_pc_s, top_rewarded_trials, N)
# plot_trajectories(fig, hs_pc_s, top_rewarded_trials, N)


In [ ]:
### colormap (bugged)
# top T trajectories binned by dot vector (avg/all)

%matplotlib qt

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.cm import ScalarMappable
from matplotlib.colors import ListedColormap

def generate_bin_vectors(N):
    angles = np.linspace(0, 2 * np.pi, N+1)[:-1]  # Omitting the last angle to avoid overlap with 0
    return np.column_stack((np.cos(angles), np.sin(angles)))

# def has_duplicates(array_2d):
#     flat_list = array_2d.flatten().tolist()
#     unique_set = set(flat_list)
#     return len(unique_set) < len(flat_list)

def generate_polar_angles(N):
    step = 360 / N
    angles = [(i * step) % 360 for i in range(N)]
    return [f"{int(angle)}°" if angle <= 180 else f"{int(angle - 360)}°" for angle in angles]

def compute_dot_direction(dot_arr):
    dot_diffs = np.diff(dot_arr, axis=1)
    mean_dot_dir = np.mean(dot_diffs, axis=1)
    norm_dot_dir = np.linalg.norm(mean_dot_dir, axis=1)
    mean_dot_dir_normalized = mean_dot_dir / norm_dot_dir[:, np.newaxis]
    return mean_dot_dir_normalized

def get_top_rewarded_trials_new(dot_arr, r_arr, T, N):
    bins = generate_bin_vectors(N)
    normalized_dot_dirs = compute_dot_direction(dot_arr)

    # Assign each dot to a bin
    bin_assignments = []
    for dot in normalized_dot_dirs:
        cosine_similarity = dot @ np.array(bins).T
        assigned_bin = np.argmax(cosine_similarity)
        bin_assignments.append(assigned_bin)

    # Organize trials by their assigned bin and then sort them by total reward
    total_rewards = np.sum(r_arr, axis=1)
    bin_lists = [[] for _ in range(N)]
    for i, assigned_bin in enumerate(bin_assignments):
        bin_lists[assigned_bin].append(i)
    
    # Sort trials in each bin by reward and select top T trials
    top_trials = np.zeros((T, N), dtype=int)
    for idx, trial_list in enumerate(bin_lists):
        sorted_trials = sorted(trial_list, key=lambda x: total_rewards[x], reverse=True)
        top_trials[:, idx] = sorted_trials[:T]

    return top_trials

def plot_trajectories(fig, data, top_trials, N):
    # Define a color wheel
    color_wheel = plt.cm.hsv(np.linspace(0, 1, N+1))
    custom_cmap = LinearSegmentedColormap.from_list('custom_cmap', color_wheel)

    clrs=['#ADD8E6', '#1E90FF', '#0000CD', '#00008B']

    custom_cmap = ListedColormap(clrs)

    # Create a ScalarMappable object with the colormap
    norm = plt.Normalize(vmin=0, vmax=len(clrs)-1)
    sm = ScalarMappable(norm=norm, cmap=custom_cmap)
    
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122)
    
    for bin_idx, trials in enumerate(top_trials.T): # Transpose to iterate over trials per bin
        for trial_idx in trials:
            traj = data[trial_idx, :, :3]
            segments_3d = [traj[i:i+2] for i in range(traj.shape[0]-1)]
            segments_2d = [s[:, :2] for s in segments_3d]
            
            lc_3d = Line3DCollection(segments_3d, colors=color_wheel[bin_idx])
            lc_2d = LineCollection(segments_2d, colors=color_wheel[bin_idx])
            
            # ax1.add_collection(lc_3d)
            ax2.add_collection(lc_2d)
    
    # Set limits based on data
    all_data = data[:, :, :3].reshape(-1, 3)
    ax1.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax1.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    ax1.set_zlim(all_data[:, 2].min(), all_data[:, 2].max())
    # ax2.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    # ax2.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())

    polar_angles = generate_polar_angles(N) # ["0°", "45°", "90°", "135°", "±180°", "-135°", "-90°", "-45°"]
    # legend_elements = [Line2D([0], [0], color=color_wheel[i], lw=2, label=polar_angles[i]) for i in range(N)]
    # ax2.legend(handles=legend_elements, loc="upper right")
    
    ax1.set_title('3D Trajectories')
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    # ax1.set_zlabel('PC3')
    ax2.set_title('2D Projections of Trajectories')
    ax2.set_xlabel('PC1')
    ax2.set_ylabel('PC2')
    
    plt.show()

def plot_avg_trajectories(fig, data, top_trials, N):
    # Define a color wheel
    color_wheel = plt.cm.hsv(np.linspace(0, 1, N+1))
    hsv_colors = plt.cm.hsv(np.linspace(0, 1, N))
    custom_cmap = mpl.colors.ListedColormap(hsv_colors)
    bounds = np.linspace(0, N, N + 1)
    norm = mpl.colors.BoundaryNorm(bounds, custom_cmap.N)
    # custom_cmap = LinearSegmentedColormap.from_list('custom_cmap', color_wheel)

    clrs=['#ADD8E6', '#1E90FF', '#0000CD', '#00008B']

    custom_cmap = ListedColormap(clrs)

    # Create a ScalarMappable object with the colormap
    norm = plt.Normalize(vmin=0, vmax=len(clrs)-1)
    sm = ScalarMappable(norm=norm, cmap=custom_cmap)
    
    # ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122)
    
    for bin_idx, trials in enumerate(top_trials.T): 
        # Calculate the average trajectory for the trials in this bin
        avg_traj = np.mean(data[trials], axis=0)
        
        segments_3d = [avg_traj[i:i+2] for i in range(avg_traj.shape[0]-1)]
        segments_2d = [s[:, :2] for s in segments_3d]
        
        lc_3d = Line3DCollection(segments_3d, colors=color_wheel[bin_idx])
        lc_2d = LineCollection(segments_2d, colors=color_wheel[bin_idx])
        
        # ax1.add_collection(lc_3d)
        # ax2.add_collection(lc_2d)
    
    top_data_indices = top_trials.flatten()  # Assuming top_trials is a 2D array of indices
    top_data = data[top_data_indices, :, :3].reshape(-1, 3)

    # Define padding as a percentage of the data range
    padding_factor = -0.0  # 5% padding

    # Calculate the limits based on the top_data with padding
    x_padding = (top_data[:, 0].max() - top_data[:, 0].min()) * padding_factor
    y_padding = (top_data[:, 1].max() - top_data[:, 1].min()) * padding_factor
    z_padding = (top_data[:, 2].max() - top_data[:, 2].min()) * padding_factor

    x_range = (top_data[:, 0].min() - x_padding, top_data[:, 0].max() + x_padding)
    y_range = (top_data[:, 1].min() - y_padding, top_data[:, 1].max() + y_padding)
    z_range = (top_data[:, 2].min() - z_padding, top_data[:, 2].max() + z_padding)

    # Set the limits for each axis based on the top_data with padding
    ax2.set_xlim(x_range)
    ax2.set_ylim(y_range)
    # ax2.set_zlim(z_range)

    sm = plt.cm.ScalarMappable(cmap=custom_cmap, norm=plt.Normalize(vmin=0, vmax=N))
    sm.set_array([])  # You can set an array of values here if needed for scaling
    cbar = plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=custom_cmap), cax=ax2, orientation='horizontal', ticks=[], aspect=10)
    cbar.outline.set_edgecolor('none')  # Remove the colorbar border
    cbar.ax.set_xticks([])  # Removes the ticks on the x-axis (for horizontal colorbar)
    cbar.ax.set_yticks([])

    # Manually adjust the size and position of the colorbar
    cbar_width = ax2.get_position().width
    cbar_height = 0.025  # Set the height of the colorbar
    cbar_pos_x = ax2.get_position().x0
    cbar_pos_y = ax2.get_position().y1 # + 0.05  # Adjust vertical position
    cbar.ax.set_position([cbar_pos_x - 0.08, cbar_pos_y - .07, cbar_width, cbar_height])

    # Remove tick marks from the colorbar
    cbar.ax.xaxis.set_ticks_position('top')
    cbar.ax.xaxis.set_tick_params(size=0)

    # Add the 'time' label to the right
    cbar.ax.text(1.1, 0.5, 'speed', verticalalignment='center', transform=cbar.ax.transAxes, fontsize=20)

    # ax1.set_box_aspect([1,1,1]) 
    # ax2.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    # ax2.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    
    # Create a legend with polar angles corresponding to the bins
    polar_angles = generate_polar_angles(N) # ["0°", "45°", "90°", "135°", "±180°", "-135°", "-90°", "-45°"]
    legend_elements = [Line2D([0], [0], color=color_wheel[i], lw=2, label=polar_angles[i]) for i in range(N)]
    # ax2.legend(handles=legend_elements, loc="upper right")
    
    # x_min, x_max = all_data[:, 0].min(), all_data[:, 0].max()
    # y_min, y_max = all_data[:, 1].min(), all_data[:, 1].max()
    # z_min, z_max = all_data[:, 2].min(), all_data[:, 2].max()

    # ax1.set_title('3D Average Trajectories')
    # ax2.set_xlabel('PC1', fontsize=fnt, labelpad=-5, rotation=0)
    ax2.set_xticks([])
    ax2.spines['top'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    ax2.spines['left'].set_linewidth(4)
    ax2.spines['bottom'].set_linewidth(4)
    # ax1.xaxis.pane.fill = False
    # ax2.set_ylabel('PC2', fontsize=fnt, labelpad=-5, rotation=0)
    ax2.set_yticks([])
    ax2.spines['top'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    ax2.spines['left'].set_linewidth(4)
    ax2.spines['bottom'].set_linewidth(4)
    # ax2.set_zlabel('PC3', fontsize=fnt, labelpad=-10, rotation=90)
    # ax2.set_zticks([])
    ax2.spines['top'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    ax2.spines['left'].set_linewidth(4)
    ax2.spines['bottom'].set_linewidth(4)
    # ax2.set_title('2D Projections of Average Trajectories')
    # ax2.set_xlabel('PC1')
    # ax2.set_ylabel('PC2')
    ax2.grid(False)
    # ax1.legend(handles=legend_elements, loc="upper right", fontsize=fnt, frameon=False, bbox_to_anchor=(1.0, 0.5))
    
    plt.show()

N = 8 # 8
T = 60
fnt = 16

# Assuming hs_pc_s data is available
fig = plt.figure(figsize=(8, 6))
top_rewarded_trials = get_top_rewarded_trials_new(dot_arr, r_arr, T, N)
print('trt=',top_rewarded_trials.shape)
plot_avg_trajectories(fig, hs_pc_s, top_rewarded_trials, N)
# plot_trajectories(fig, hs_pc_s, top_rewarded_trials, N)


In [ ]:
# legend misc

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm

# Assuming you have an array of angles and corresponding colors
angles = np.linspace(0, 360, 360)  # Example angles from 0 to 360 degrees
colors = plt.cm.hsv(angles / 360.)  # Corresponding colors using the HSV colormap

# Create a figure and a subplot
fig, ax = plt.subplots()

# Create a LineCollection with a colormap
cmap = ListedColormap(colors)
norm = BoundaryNorm(angles, cmap.N)
points = np.array([np.linspace(0, 1, 100), np.linspace(0, 0, 100)]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)
lc = LineCollection(segments, cmap=cmap, norm=norm, linewidth=2)
lc.set_array(angles)

# Add LineCollection to the axes for the legend
ax.add_collection(lc)

# Create a custom legend entry
from matplotlib.legend_handler import HandlerLineCollection
class HandlerColormap(HandlerLineCollection):
    def __init__(self, cmap, num_stripes=10, **kw):
        HandlerLineCollection.__init__(self, **kw)
        self.cmap = cmap
        self.num_stripes = num_stripes
    def create_artists(self, legend, orig_handle, xdescent, ydescent, width, height, fontsize, trans):
        stripes = [self.cmap(i * 1.0 / self.num_stripes) for i in range(self.num_stripes)]
        lc = LineCollection([[(xdescent, ydescent + height / 2), (xdescent + width, ydescent + height / 2)]], 
                            colors=stripes, linewidth=orig_handle.get_linewidth())
        return [lc]

# Add legend to the plot
plt.legend([lc], ['Angle'], handler_map={lc: HandlerColormap(cmap)}, handlelength=2)

# Display the plot
plt.show()


In [ ]:
# trajectories binned by dot vector - random (avg/all)
# 

%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d.art3d import Line3DCollection

def generate_bin_vectors(N):
    angles = np.linspace(0, 2 * np.pi, N+1)[:-1]  # Omitting the last angle to avoid overlap with 0
    return np.column_stack((np.cos(angles), np.sin(angles)))

def generate_polar_angles(N):
    step = 360 / N
    angles = [(i * step) % 360 for i in range(N)]
    return [f"{int(angle)}°" if angle <= 180 else f"{int(angle - 360)}°" for angle in angles]

def compute_dot_direction(dot_arr):
    dot_diffs = np.diff(dot_arr, axis=1)
    mean_dot_dir = np.mean(dot_diffs, axis=1)
    norm_dot_dir = np.linalg.norm(mean_dot_dir, axis=1)
    mean_dot_dir_normalized = mean_dot_dir / norm_dot_dir[:, np.newaxis]
    return mean_dot_dir_normalized

def get_binned_trials(dot_arr, N):
    bins = generate_bin_vectors(N)
    normalized_dot_dirs = compute_dot_direction(dot_arr)

    # Assign each dot to a bin
    bin_assignments = []
    for dot in normalized_dot_dirs:
        cosine_similarity = dot @ np.array(bins).T
        assigned_bin = np.argmax(cosine_similarity)
        bin_assignments.append(assigned_bin)

    # Organize trials by their assigned bin
    bin_lists = [[] for _ in range(N)]
    for i, assigned_bin in enumerate(bin_assignments):
        bin_lists[assigned_bin].append(i)

    # Find the bin with the maximum number of trials
    max_bin_length = max(len(trial_list) for trial_list in bin_lists)
    
    # Preallocate an array filled with nans
    binned_trials = np.full((max_bin_length, N), np.nan)
    
    # Fill the array with trial indices for each bin
    for idx, trial_list in enumerate(bin_lists):
        binned_trials[:len(trial_list), idx] = trial_list

    return binned_trials

def plot_trajectories(fig, data, top_trials, N, T):
    # define a color wheel
    color_wheel = plt.cm.hsv(np.linspace(0, 1, N+1))
    
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122)
    
    for bin_idx, trials in enumerate(top_trials.T): # Transpose to iterate over trials per bin
        valid_trials = trials[~np.isnan(trials)].astype(int)  # Remove nan values and convert to int
        
        # select T random trials from valid trials
        selected_trials = np.random.choice(valid_trials, min(T, len(valid_trials)), replace=False)
        
        for trial_idx in selected_trials:
            traj = data[trial_idx, :, :3]
            segments_3d = [traj[i:i+2] for i in range(traj.shape[0]-1)]
            segments_2d = [s[:, 1:3] for s in segments_3d]
            
            lc_3d = Line3DCollection(segments_3d, colors=color_wheel[bin_idx])
            lc_2d = LineCollection(segments_2d, colors=color_wheel[bin_idx])
            
            ax1.add_collection(lc_3d)
            ax2.add_collection(lc_2d)
    
    # Set limits based on data
    all_data = data[:, :, :3].reshape(-1, 3)
    ax1.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax1.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    ax1.set_zlim(all_data[:, 2].min(), all_data[:, 2].max())
    ax2.set_xlim(all_data[:, 1].min(), all_data[:, 1].max())
    ax2.set_ylim(all_data[:, 2].min(), all_data[:, 2].max())

    polar_angles = generate_polar_angles(N) # ["0°", "45°", "90°", "135°", "±180°", "-135°", "-90°", "-45°"]
    legend_elements = [Line2D([0], [0], color=color_wheel[i], lw=2, label=polar_angles[i]) for i in range(N)]
    ax2.legend(handles=legend_elements, loc="upper right")
    
    ax1.set_title('3D Trajectories')
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    ax1.set_zlabel('PC3')
    ax2.set_title('2D Projections of Trajectories')
    ax2.set_xlabel('PC2')
    ax2.set_ylabel('PC3')
    
    plt.show()

def plot_avg_trajectories(fig, data, top_trials, N):
    # define a color wheel
    color_wheel = plt.cm.hsv(np.linspace(0, 1, N+1))
    
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122)
    
    # for bin_idx, trials in enumerate(top_trials.T):
    print('top_trials=',top_trials.shape)
    k = 6
    bin_idx = k
    trials = top_trials.T[k]
    valid_trials = trials[~np.isnan(trials)].astype(int)  # Remove nan values and convert to int
    # calculate the average trajectory for the trials in this bin
    avg_traj = np.mean(data[valid_trials], axis=0)
    
    segments_3d = [avg_traj[i:i+2] for i in range(avg_traj.shape[0]-1)]
    segments_2d = [s[:, 1:3] for s in segments_3d]
    
    lc_3d = Line3DCollection(segments_3d, colors=color_wheel[bin_idx])
    lc_2d = LineCollection(segments_2d, colors=color_wheel[bin_idx])
    
    ax1.add_collection(lc_3d)
    ax2.add_collection(lc_2d)
    
    all_data = data[:, :, :3].reshape(-1, 3)
    ax1.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax1.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    ax1.set_zlim(all_data[:, 2].min(), all_data[:, 2].max())
    # ax2.set_xlim(all_data[:, 1].min(), all_data[:, 1].max())
    # ax2.set_ylim(all_data[:, 2].min(), all_data[:, 2].max())

    top_data_indices = trials.flatten()  # Assuming top_trials is a 2D array of indices
    top_data = data[top_data_indices.astype(int), :, :3].reshape(-1, 3)

    # Define padding as a percentage of the data range
    padding_factor = -0.3  # 5% padding

    # Calculate the limits based on the top_data with padding
    x_padding = (top_data[:, 0].max() - top_data[:, 0].min()) * padding_factor
    y_padding = (top_data[:, 1].max() - top_data[:, 1].min()) * padding_factor
    z_padding = (top_data[:, 2].max() - top_data[:, 2].min()) * padding_factor

    x_range = (top_data[:, 0].min() - x_padding, top_data[:, 0].max() + x_padding)
    y_range = (top_data[:, 1].min() - y_padding, top_data[:, 1].max() + y_padding)
    z_range = (top_data[:, 2].min() - z_padding, top_data[:, 2].max() + z_padding)

    # Set the limits for each axis based on the top_data with padding
    ax1.set_xlim(x_range)
    ax1.set_ylim(y_range)
    ax1.set_zlim(z_range)
    
    polar_angles = generate_polar_angles(N) # ["0°", "45°", "90°", "135°", "±180°", "-135°", "-90°", "-45°"]
    legend_elements = [Line2D([0], [0], color=color_wheel[i], lw=2, label=polar_angles[i]) for i in range(N)]
    # ax2.legend(handles=legend_elements, loc="upper right")
    
    ax1.set_xlabel('PC1', fontsize=fnt, labelpad=-5, rotation=0)
    ax1.set_xticks([])
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['left'].set_linewidth(4)
    ax1.spines['bottom'].set_linewidth(4)
    # ax1.xaxis.pane.fill = False
    ax1.set_ylabel('PC2', fontsize=fnt, labelpad=-5, rotation=0)
    ax1.set_yticks([])
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['left'].set_linewidth(4)
    ax1.spines['bottom'].set_linewidth(4)
    ax1.set_zlabel('PC3', fontsize=fnt, labelpad=-5, rotation=90)
    ax1.set_zticks([])
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['left'].set_linewidth(4)
    ax1.spines['bottom'].set_linewidth(4)
    ax1.grid(False)
    
    plt.show()

N = 10
# T = 110
fnt = 18

fig = plt.figure(figsize=(8, 6))
# top_rewarded_trials = get_random_trials(dot_arr, T, N)
top_rewarded_trials = get_binned_trials(dot_arr, N)
print('trt=',top_rewarded_trials.shape)
plot_avg_trajectories(fig, hs_pc_s, top_rewarded_trials, N)
# plot_avg_trajectories(fig, hv_pc_s, top_rewarded_trials, N)

# plot_trajectories(fig, hs_pc_s, top_rewarded_trials, N, T)


In [ ]:
### single trial
### new vers

%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d.art3d import Line3DCollection

def generate_bin_vectors(N):
    angles = np.linspace(0, 2 * np.pi, N+1)[:-1]
    return np.column_stack((np.cos(angles), np.sin(angles)))

def generate_polar_angles(N):
    step = 360 / N
    angles = [(i * step) % 360 for i in range(N)]
    return [f"{int(angle)}°" if angle <= 180 else f"{int(angle - 360)}°" for angle in angles]

def compute_dot_direction(dot_arr):
    dot_diffs = np.diff(dot_arr, axis=1)
    mean_dot_dir = np.mean(dot_diffs, axis=1)
    norm_dot_dir = np.linalg.norm(mean_dot_dir, axis=1)
    mean_dot_dir_normalized = mean_dot_dir / norm_dot_dir[:, np.newaxis]
    return mean_dot_dir_normalized

def get_binned_trials(dot_arr, N):
    bins = generate_bin_vectors(N)
    normalized_dot_dirs = compute_dot_direction(dot_arr)
    bin_assignments = []
    for dot in normalized_dot_dirs:
        cosine_similarity = dot @ bins.T
        assigned_bin = np.argmax(cosine_similarity)
        bin_assignments.append(assigned_bin)
    bin_lists = [[] for _ in range(N)]
    for i, assigned_bin in enumerate(bin_assignments):
        bin_lists[assigned_bin].append(i)
    max_bin_length = max(len(trial_list) for trial_list in bin_lists)
    binned_trials = np.full((max_bin_length, N), np.nan)
    for idx, trial_list in enumerate(bin_lists):
        binned_trials[:len(trial_list), idx] = trial_list
    return binned_trials

def plot_avg_trajectories(fig, data, top_trials, N):
    color_map = plt.cm.get_cmap('seismic')
    ax1 = fig.add_subplot(111, projection='3d')
    # ax2 = fig.add_subplot(122)
    trial_idx = int(top_trials[0, 0])  # Example: Select the first trial for plotting

    k = 5
    bin_idx = k
    trials = top_trials.T[k]
    valid_trials = trials[~np.isnan(trials)].astype(int)  # Remove nan values and convert to int
    # calculate the average trajectory for the trials in this bin
    avg_traj = np.mean(data[valid_trials], axis=0)

    traj = data[trial_idx, :, :3]
    num_points = traj.shape[0]
    angle_values = np.linspace(0, 1, num_points)

    segments = [avg_traj[i:i+2] for i in range(num_points - 1)]
    lc = Line3DCollection(segments, cmap=color_map, norm=plt.Normalize(0, 1))
    lc.set_array(angle_values)
    ax1.add_collection3d(lc)

    top_data_indices = trials.flatten()  # Assuming top_trials is a 2D array of indices
    top_data = data[top_data_indices.astype(int), :, :3].reshape(-1, 3)

    # Define padding as a percentage of the data range
    padding_factor = -0.29  # 5% padding

    # Calculate the limits based on the top_data with padding
    x_padding = (top_data[:, 0].max() - top_data[:, 0].min()) * padding_factor
    y_padding = (top_data[:, 1].max() - top_data[:, 1].min()) * padding_factor
    z_padding = (top_data[:, 2].max() - top_data[:, 2].min()) * padding_factor

    x_range = (top_data[:, 0].min() - x_padding, top_data[:, 0].max() + x_padding)
    y_range = (top_data[:, 1].min() - y_padding, top_data[:, 1].max() + y_padding)
    z_range = (top_data[:, 2].min() - z_padding, top_data[:, 2].max() + z_padding)

    # Set the limits for each axis based on the top_data with padding
    ax1.set_xlim(x_range)
    ax1.set_ylim(y_range)
    ax1.set_zlim(z_range)
    cbar = fig.colorbar(lc, ax=ax1, orientation='horizontal', pad=0.1, ticks=[], aspect=4)
    cbar.outline.set_edgecolor('none')

    # Manually adjust the size and position of the colorbar
    cbar_width = ax1.get_position().width
    cbar_height = 0.025  # Set the height of the colorbar
    cbar_pos_x = ax1.get_position().x0
    cbar_pos_y = ax1.get_position().y1 # + 0.05  # Adjust vertical position
    cbar.ax.set_position([cbar_pos_x - .05, cbar_pos_y - .04, cbar_width, cbar_height])

    # Remove tick marks from the colorbar
    cbar.ax.xaxis.set_ticks_position('top')
    cbar.ax.xaxis.set_tick_params(size=0)

    # Add the 'time' label to the right
    cbar.ax.text(1.2, 0.5, 'time', verticalalignment='center', transform=cbar.ax.transAxes, fontsize=14)

    ax1.set_xlabel('PC1', fontsize=fnt, labelpad=-5, rotation=0)
    ax1.set_xticks([])
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['left'].set_linewidth(4)
    ax1.spines['bottom'].set_linewidth(4)
    # ax1.xaxis.pane.fill = False
    ax1.set_ylabel('PC2', fontsize=fnt, labelpad=-5, rotation=0)
    ax1.set_yticks([])
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['left'].set_linewidth(4)
    ax1.spines['bottom'].set_linewidth(4)
    ax1.set_zlabel('PC3', fontsize=fnt, labelpad=-10, rotation=90)
    ax1.set_zticks([])
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['left'].set_linewidth(4)
    ax1.spines['bottom'].set_linewidth(4)
    ax1.grid(False)
    plt.show()

# Replace with your actual data
# dot_arr = ...
# hs_pc_s = ...

N = 10
fnt = 18

fig = plt.figure(figsize=(4, 6))
top_rewarded_trials = get_binned_trials(dot_arr, N)
plot_avg_trajectories(fig, hs_pc_s, top_rewarded_trials, N)


In [ ]:
### dot speed
### traj binned by dot speed (avg)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.colors import ListedColormap
from matplotlib.cm import ScalarMappable

# clrs=['#ADD8E6','#1E90FF','#00008B']
clrs=['#ADD8E6', '#1E90FF', '#0000CD', '#00008B']

# cmap = ListedColormap(clrs)

# # Create a ScalarMappable object with the colormap
# norm = plt.Normalize(vmin=0, vmax=len(clrs)-1)
# sm = ScalarMappable(norm=norm, cmap=cmap)

def generate_bin_vectors(N):
    angles = np.linspace(0, 2 * np.pi, N+1)[:-1]
    return np.column_stack((np.cos(angles), np.sin(angles)))

def compute_dot_speed(dot_arr):
    speeds_per_timestep = np.linalg.norm(dot_arr, axis=2)
    avg_speeds_per_trial = np.mean(speeds_per_timestep, axis=1)
    return avg_speeds_per_trial

def bin_by_dot_speed(dot_speeds, sbins):
    bin_edges = np.linspace(dot_speeds.min(), dot_speeds.max(), sbins + 1)
    bin_indices = np.digitize(dot_speeds, bin_edges) - 1
    return bin_indices

def get_avg_trajectories(data, bin_indices, sbins):
    avg_trajectories = []
    for i in range(sbins):
        avg_trajectories.append(np.mean(data[bin_indices == i], axis=0))
    return avg_trajectories

def plot_avg_trajectories_by_speed(fig, data, dot_speeds, sbins):
    color_wheel = plt.cm.jet(np.linspace(0, 1, sbins))
    
    bin_indices = bin_by_dot_speed(dot_speeds, sbins)

    avg_trajectories = get_avg_trajectories(data, bin_indices, sbins)
    
    ax1 = fig.add_subplot(121, projection='3d')
    # ax2 = fig.add_subplot(122)
    
    for bin_idx, avg_traj in enumerate(avg_trajectories): 
        print('bin_idx=',bin_idx)
        segments_3d = [avg_traj[i:i+2] for i in range(avg_traj.shape[0]-1)]
        segments_2d = [s[:, :2] for s in segments_3d]
        
        lc_3d = Line3DCollection(segments_3d, colors=clrs[bin_idx])
        # lc_2d = LineCollection(segments_2d, colors=clrs[bin_idx])
        
        ax1.add_collection(lc_3d)
        # ax2.add_collection(lc_2d)
    
    all_data = data.reshape(-1, 3)
    ax1.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax1.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    ax1.set_zlim(all_data[:, 2].min(), all_data[:, 2].max())
    # ax2.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    # ax2.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    
    bin_labels = ["speed " + str(i+1) for i in range(sbins)]
    legend_elements = [Line2D([0], [0], color=clrs[i], lw=2, label=bin_labels[i]) for i in range(sbins)]
    
    # ax1.set_title('3D Average Trajectories by Dot Speed')
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    ax1.set_zlabel('PC3')
    # ax2.set_title('2D Projections of Average Trajectories by Dot Speed')
    # ax2.set_xlabel('PC1')
    # ax2.set_ylabel('PC2')

    ax1.set_xlabel('PC1', fontsize=fnt, labelpad=-5, rotation=0)
    ax1.set_xticks([])
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['left'].set_linewidth(4)
    ax1.spines['bottom'].set_linewidth(4)
    # ax1.xaxis.pane.fill = False
    ax1.set_ylabel('PC2', fontsize=fnt, labelpad=-0, rotation=0)
    ax1.set_yticks([])
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['left'].set_linewidth(4)
    ax1.spines['bottom'].set_linewidth(4)
    ax1.set_zlabel('PC3', fontsize=fnt, labelpad=-10, rotation=90)
    ax1.set_zticks([])
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['left'].set_linewidth(4)
    ax1.spines['bottom'].set_linewidth(4)
    ax1.grid(False)

    ax1.legend(handles=legend_elements, loc="upper center", ncol=4, fontsize=14, frameon=False, handlelength=1.5, columnspacing=0.5, bbox_to_anchor=(0.53, 1.05))
    
    plt.show()

sbins = 4
# Assuming hs_pc_s data and dot_arr are available
fnt = 18

dot_speeds = compute_dot_speed(dot_arr)
fig = plt.figure(figsize=(8, 6))
plot_avg_trajectories_by_speed(fig, hs_pc_s, dot_speeds, sbins)


In [ ]:
# speed binned, single direction (avg)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d.art3d import Line3DCollection

def compute_dot_direction(dot_arr):
    dot_diffs = np.diff(dot_arr, axis=1)
    mean_dot_dir = np.mean(dot_diffs, axis=1)
    norm_dot_dir = np.linalg.norm(mean_dot_dir, axis=1)
    mean_dot_dir_normalized = mean_dot_dir / norm_dot_dir[:,np.newaxis]
    return mean_dot_dir_normalized

def bin_by_dot_direction(dot_directions, bins):
    cosine_similarity = dot_directions @ np.array(bins).T
    bin_indices = np.argmax(cosine_similarity, axis=1)
    return bin_indices

def compute_dot_speed(dot_arr):
    dot_diffs = np.diff(dot_arr, axis=1)
    speeds = np.linalg.norm(dot_diffs, axis=2)
    avg_speeds = np.mean(speeds, axis=1)
    return avg_speeds

def bin_by_speed(dot_speeds, sbins):
    bin_edges = np.linspace(dot_speeds.min(), dot_speeds.max(), sbins + 1)
    bin_indices = np.digitize(dot_speeds, bin_edges) - 1
    return bin_indices

def plot_avg_trajectories_by_direction_and_speed(fig, data, dot_directions, bins, dbin, dot_speeds, sbins):
    # Define a color wheel
    color_wheel = plt.cm.jet(np.linspace(0, 1, sbins))
    
    # Compute the binned indices for directions and speeds
    direction_indices = bin_by_dot_direction(dot_directions, bins)
    speed_indices = bin_by_speed(dot_speeds, sbins)

    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122)
    
    # For the chosen direction bin, further bin by speed and plot averaged trajectories
    for speed_bin_idx in range(sbins):
        selected_indices = np.logical_and(direction_indices == dbin, speed_indices == speed_bin_idx)
        avg_traj = np.mean(data[selected_indices], axis=0)
        
        segments_3d = [avg_traj[i:i+2] for i in range(avg_traj.shape[0]-1)]
        segments_2d = [s[:, :2] for s in segments_3d]
        
        lc_3d = Line3DCollection(segments_3d, colors=color_wheel[speed_bin_idx])
        lc_2d = LineCollection(segments_2d, colors=color_wheel[speed_bin_idx])
        
        ax1.add_collection(lc_3d)
        ax2.add_collection(lc_2d)
    
    # Set limits based on data
    all_data = data.reshape(-1, 3)
    ax1.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax1.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    ax1.set_zlim(all_data[:, 2].min(), all_data[:, 2].max())
    ax2.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax2.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    
    # Create a legend based on speed bins
    bin_labels = ["Speed Bin " + str(i+1) for i in range(sbins)]
    legend_elements = [Line2D([0], [0], color=color_wheel[i], lw=2, label=bin_labels[i]) for i in range(sbins)]
    
    ax2.legend(handles=legend_elements, loc="upper right")
    
    ax1.set_title(f'3D Average Trajectories by Speed (Direction Bin {dbin})')
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    ax1.set_zlabel('PC3')
    ax2.set_title(f'2D Projections of Average Trajectories by Speed (Direction Bin {dbin})')
    ax2.set_xlabel('PC1')
    ax2.set_ylabel('PC2')
    
    plt.show()

N = 8
bins = generate_bin_vectors(N)
dot_directions = compute_dot_direction(dot_arr)
dot_speeds = compute_dot_speed(dot_arr)
dbin = 4
sbins = 3
fig = plt.figure(figsize=(12, 6))
# plot_avg_trajectories_by_direction_and_speed(fig, hs_pc_s, dot_directions, bins, dbin, dot_speeds, sbins)
plot_avg_trajectories_by_direction_and_speed(fig, hv_pc_s, dot_directions, bins, dbin, dot_speeds, sbins)

In [ ]:
# speed, direction binned all (avg)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d.art3d import Line3DCollection

def generate_bin_vectors(N):
    angles = np.linspace(0, 2 * np.pi, N+1)[:-1]
    return np.column_stack((np.cos(angles), np.sin(angles)))

def compute_dot_direction(dot_arr):
    dot_diffs = np.diff(dot_arr, axis=1)
    mean_dot_dir = np.mean(dot_diffs, axis=1)
    norm_dot_dir = np.linalg.norm(mean_dot_dir, axis=1)
    mean_dot_dir_normalized = mean_dot_dir / norm_dot_dir[:,np.newaxis]
    return mean_dot_dir_normalized

def bin_by_dot_direction(dot_directions, bins):
    cosine_similarity = dot_directions @ np.array(bins).T
    bin_indices = np.argmax(cosine_similarity, axis=1)
    return bin_indices

def compute_dot_speed(dot_arr):
    dot_diffs = np.diff(dot_arr, axis=1)
    speeds = np.linalg.norm(dot_diffs, axis=2)
    avg_speeds = np.mean(speeds, axis=1)
    return avg_speeds

def bin_by_speed(dot_speeds, sbins):
    bin_edges = np.linspace(dot_speeds.min(), dot_speeds.max(), sbins + 1)
    bin_indices = np.digitize(dot_speeds, bin_edges) - 1
    return bin_indices

def plot_avg_trajectories_by_all_directions(fig, data, dot_directions, bins, dot_speeds, sbins):
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122)
    
    direction_indices = bin_by_dot_direction(dot_directions, bins)
    speed_indices = bin_by_speed(dot_speeds, sbins)

    # hsv colormap based on direction
    hue_values = np.linspace(0, 1, len(bins) + 1)[:-1]

    for dbin, hue in zip(range(len(bins)), hue_values):
        for speed_bin_idx in range(sbins):
            selected_indices = np.logical_and(direction_indices == dbin, speed_indices == speed_bin_idx)
            avg_traj = np.mean(data[selected_indices], axis=0)
            
            segments_3d = [avg_traj[i:i+2] for i in range(avg_traj.shape[0]-1)]
            segments_2d = [s[:, :2] for s in segments_3d]
            
            # darker color for faster speed bins
            color = hsv_to_rgb((hue, 1, 1 - speed_bin_idx / (sbins + 1)))
            
            lc_3d = Line3DCollection(segments_3d, colors=color)
            lc_2d = LineCollection(segments_2d, colors=color)
            
            ax1.add_collection(lc_3d)
            ax2.add_collection(lc_2d)
            
            # optional, plot grey dot at the 15th timestep
            # ax1.scatter(avg_traj[14, 0], avg_traj[14, 1], avg_traj[14, 2], color='grey', s=30)
            # ax2.plot(avg_traj[14, 0], avg_traj[14, 1], 'o', color='grey', markersize=4)

    # Set limits
    all_data = data.reshape(-1, 3)
    ax1.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax1.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    ax1.set_zlim(all_data[:, 2].min(), all_data[:, 2].max())
    ax2.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax2.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())

    ax1.set_title('3D Average Trajectories by Direction and Speed')
    ax2.set_title('2D Projections of Average Trajectories by Direction and Speed')
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    ax1.set_zlabel('PC3')
    ax2.set_xlabel('PC1')
    ax2.set_ylabel('PC2')

    # Legend on the ax2 subplot
    legend_elements = [Line2D([0], [0], color=hsv_to_rgb((hue, 1, 0.5)), lw=2, label=f"Direction Bin {dbin}") for dbin, hue in zip(range(len(bins)), hue_values)]
    ax2.legend(handles=legend_elements, loc="upper right")
    
    plt.tight_layout()
    plt.show()

N = 3
bins = generate_bin_vectors(N)
dot_directions = compute_dot_direction(dot_arr)
dot_speeds = compute_dot_speed(dot_arr)
sbins = 2

fig = plt.figure(figsize=(12, 6))  
# plot_avg_trajectories_by_all_directions(fig, hs_pc_s, dot_directions, bins, dot_speeds, sbins)
plot_avg_trajectories_by_all_directions(fig, hv_pc_s, dot_directions, bins, dot_speeds, sbins)


In [ ]:
# speed, direction binned (old/doesnt work)

import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d.art3d import Line3DCollection

def generate_bin_vectors(N):
    angles = np.linspace(0, 2 * np.pi, N+1)[:-1]
    return np.column_stack((np.cos(angles), np.sin(angles)))

def compute_dot_direction(dot_arr):
    dot_diffs = np.diff(dot_arr, axis=1)
    mean_dot_dir = np.mean(dot_diffs, axis=1)
    norm_dot_dir = np.linalg.norm(mean_dot_dir, axis=1)
    mean_dot_dir_normalized = mean_dot_dir / norm_dot_dir[:, np.newaxis]
    return mean_dot_dir_normalized

def compute_speed(dot_arr):
    diff = np.diff(dot_arr, axis=1)
    speed = np.linalg.norm(diff, axis=2)
    avg_speed = np.mean(speed, axis=1)
    return avg_speed

def get_binned_indices(dot_arr, dbins, sbins):
    bins = generate_bin_vectors(dbins)
    dot_dirs = compute_dot_direction(dot_arr)
    speeds = compute_speed(dot_arr)
    speed_bins = np.linspace(np.min(speeds), np.max(speeds), sbins+1)
    
    bin_indices = {}
    
    for i, dot in enumerate(dot_dirs):
        # Find dot vector bin
        cosine_similarity = dot @ bins.T
        d_bin = np.argmax(cosine_similarity)

        # Find speed bin
        s_bin = np.digitize(speeds[i], speed_bins) - 1

        if (d_bin, s_bin) not in bin_indices:
            bin_indices[(d_bin, s_bin)] = []
        bin_indices[(d_bin, s_bin)].append(i)
    
    return bin_indices

def plot_avg_trajectories_by_bins(fig, data, dot_arr, dbins, sbins):   
    avg_dot_vectors = np.mean(np.linalg.norm(dot_arr, axis=-1), axis=-1)
    d_bin_edges = np.linspace(avg_dot_vectors.min(), avg_dot_vectors.max(), dbins+1)

    # Calculate speed for each trial as the mean speed over timesteps
    speeds = np.linalg.norm(np.diff(data, axis=1), axis=2).mean(axis=1)

    bin_indices = defaultdict(list)
    s_bin_edges_list = []

    for d in range(dbins):
        d_mask = (dot_arr >= d_bin_edges[d]) & (dot_arr < d_bin_edges[d+1])
        
        if np.sum(d_mask) == 0:  # No data points in this dot bin
            s_bin_edges_list.append(np.linspace(0, 1, sbins+1))  # Dummy values
            continue
        
        relevant_speeds = speeds[d_mask]
        s_bin_edges = np.linspace(relevant_speeds.min(), relevant_speeds.max(), sbins+1)
        s_bin_edges_list.append(s_bin_edges)
        
        for s in range(sbins):
            s_mask = (speeds >= s_bin_edges[s]) & (speeds < s_bin_edges[s+1])
            combined_mask = d_mask & s_mask
            bin_indices[(d, s)].extend(np.where(combined_mask)[0])
            
    ax = fig.add_subplot(1, 1, 1)
    color_wheel = plt.cm.jet(np.linspace(0, 1, dbins*sbins))
    
    for idx, trials in enumerate(bin_indices.values()):
        avg_traj = np.mean(data[trials], axis=0)
        ax.plot(avg_traj[:, 0], avg_traj[:, 1], color=color_wheel[idx])
        
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title('Averaged Trajectories Binned by Dot Vector and Speed')
    plt.show()

dbins = 8
sbins = 2

# Usage example (assuming you've loaded or generated 'dot_arr' and 'hs_pc_s'):
fig = plt.figure(figsize=(12, 6))
plot_avg_trajectories_by_bins(fig, hs_pc_s, dot_arr, dbins, sbins)


In [ ]:
# speed, direction binned (old/doesnt work)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d.art3d import Line3DCollection

def compute_dot_speed(dot_arr):
    """Compute the average dot speed across timesteps for each trial."""
    dot_diffs = np.diff(dot_arr, axis=1)
    mean_dot_vel = np.mean(dot_diffs, axis=1)
    mean_dot_speeds = np.linalg.norm(mean_dot_vel, axis=1)
    return mean_dot_speeds

def bin_by_direction(dot_directions, dbins):
    bin_edges = np.linspace(-1, 1, dbins + 1)  # Assuming directions range from -1 to 1
    bin_indices = np.digitize(dot_directions, bin_edges) - 1
    return bin_indices

def bin_by_speed(dot_speeds, sbins):
    bin_edges = np.linspace(dot_speeds.min(), dot_speeds.max() + 1e-10, sbins + 1)
    bin_indices = np.digitize(dot_speeds, bin_edges) - 1
    return bin_indices

def plot_avg_trajectories_by_direction_and_speed(fig, data, dot_directions_bins, dbins, dot_speeds, sbins):
    # Define primary colors for the direction bins
    primary_colors = plt.cm.tab10(np.linspace(0, 1, dbins))
    
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122)
    
    legend_elements = []
    
    for dir_bin_idx in range(dbins):
        # Get shades for speed bins within the primary color
        shades = [primary_colors[dir_bin_idx] * (0.5 + 0.5 * (i+1)/sbins) for i in range(sbins)]
        
        # Create legend for current direction bin
        for speed_bin_idx in range(sbins):
            legend_elements.append(Line2D([0], [0], color=shades[speed_bin_idx], lw=2, 
                                        label=f"Direction {dir_bin_idx + 1}, Speed Bin {speed_bin_idx + 1}"))
        
        for speed_bin_idx in range(sbins):
            selected_indices = np.logical_and(dot_directions_bins[:, 0] == dir_bin_idx, 
                                            bin_by_speed(dot_speeds, sbins) == speed_bin_idx)
            
            # Debugging print statement
            print(f"Direction {dir_bin_idx + 1}, Speed Bin {speed_bin_idx + 1}: {selected_indices.sum()} selected trials")
            
            avg_traj = np.mean(data[selected_indices], axis=0)
            
            # Debugging print statement
            if np.isnan(avg_traj).any():
                print(f"Direction {dir_bin_idx + 1}, Speed Bin {speed_bin_idx + 1} has NaN values in the averaged trajectory.")
            
            segments_3d = [avg_traj[i:i+2] for i in range(avg_traj.shape[0]-1)]
            segments_2d = [s[:, :2] for s in segments_3d]
            
            lc_3d = Line3DCollection(segments_3d, colors=shades[speed_bin_idx])
            lc_2d = LineCollection(segments_2d, colors=shades[speed_bin_idx])
            
            ax1.add_collection(lc_3d)
            ax2.add_collection(lc_2d)
    
    # Set limits based on data
    all_data = data.reshape(-1, 3)
    ax1.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax1.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    ax1.set_zlim(all_data[:, 2].min(), all_data[:, 2].max())
    ax2.set_xlim(all_data[:, 0].min(), all_data[:, 0].max())
    ax2.set_ylim(all_data[:, 1].min(), all_data[:, 1].max())
    
    ax2.legend(handles=legend_elements, loc="upper right")
    
    ax1.set_title('3D Average Trajectories by Direction and Speed')
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    ax1.set_zlabel('PC3')
    ax2.set_title('2D Projections of Average Trajectories by Direction and Speed')
    ax2.set_xlabel('PC1')
    ax2.set_ylabel('PC2')
    
    plt.tight_layout()
    plt.show()

# Assuming your bins and hs_pc_s data are available
dbins = 8
sbins = 2

dot_directions = compute_dot_direction(dot_arr)
dot_directions_bins = bin_by_direction(dot_directions, dbins)
dot_speeds = compute_dot_speed(dot_arr)
fig = plt.figure(figsize=(14, 7))
plot_avg_trajectories_by_direction_and_speed(fig, hs_pc_s, dot_directions_bins, dbins, dot_speeds, sbins)


In [ ]:
# debug (hist)

import numpy as np
import matplotlib.pyplot as plt

# Assuming your dot_arr is defined and normalized
norms = np.linalg.norm(dot_arr, axis=2)
normalized_dot_arr = dot_arr / norms[:, :, np.newaxis]

# Compute the angles of these vectors in degrees
angles = np.arctan2(normalized_dot_arr[:,:,1], normalized_dot_arr[:,:,0]) 
angles_deg = np.degrees(angles) % 360  # Convert to degrees and ensure the range is [0, 360)

# Flatten the angles array for the histogram
angles_flat = angles_deg.flatten()

# Plot histogram
plt.figure(figsize=(10,6))
plt.hist(angles_flat, bins=360, range=(0,360), color='blue', alpha=0.7)
plt.title("Histogram of Directions of Normalized Dot Vectors")
plt.xlabel("Direction (degrees)")
plt.ylabel("Number of Dot Vectors")
plt.grid(True)
plt.show()


In [ ]:
# debug (scatter)

import numpy as np
import matplotlib.pyplot as plt

# Assuming your bins are defined elsewhere
bins = generate_bin_vectors(N)

# Calculate the average dot velocity vectors
dot_diffs = np.diff(dot_arr, axis=1)
mean_dot_vel = np.mean(dot_diffs, axis=1)
norm_dot_vel = np.linalg.norm(mean_dot_vel, axis=1)
normalized_dot_vel = mean_dot_vel / norm_dot_vel[:, np.newaxis]

# Assign each normalized dot velocity vector to a bin
bin_assignments = []
for dot in normalized_dot_vel:
    cosine_similarity = dot @ np.array(bins).T
    assigned_bin = np.argmax(cosine_similarity)
    bin_assignments.append(assigned_bin)

# Scatter plot each bin
plt.figure(figsize=(10,10))
for idx, bin_val in enumerate(bins):
    selected_dots = normalized_dot_vel[np.array(bin_assignments) == idx] # normalized_dot_vel[np.array(bin_assignments) == idx]
    print('idx=',idx,'selected_dots.shape=',selected_dots.shape)
    plt.scatter(selected_dots[:, 0], selected_dots[:, 1], label=f'Bin {idx}')

plt.legend()
plt.title("Spread of normalized dot velocity vectors for each bin")
plt.xlabel("X-coordinate")
plt.ylabel("Y-coordinate")
plt.grid(True)
plt.show()

print('bin_assignments=',bin_assignments)

In [ ]:
# debug (scatter 2)

import numpy as np
import matplotlib.pyplot as plt

def generate_bin_vectors(N):
    angles = np.linspace(0, 2 * np.pi, N+1)[:-1]  # Omitting the last angle to avoid overlap with 0
    return np.column_stack((np.cos(angles), np.sin(angles)))

def compute_dot_direction(dot_arr):
    dot_diffs = np.diff(dot_arr, axis=1)
    mean_dot_dir = np.mean(dot_diffs, axis=1)
    norm_dot_dir = np.linalg.norm(mean_dot_dir, axis=1)
    mean_dot_dir_normalized = mean_dot_dir / norm_dot_dir[:, np.newaxis]
    return mean_dot_dir_normalized

def get_top_rewarded_trials_new(dot_arr, r_arr, T, N):
    bins = generate_bin_vectors(N)
    normalized_dot_dirs = compute_dot_direction(dot_arr)

    # Assign each dot to a bin
    bin_assignments = []
    for dot in normalized_dot_dirs:
        cosine_similarity = dot @ np.array(bins).T
        assigned_bin = np.argmax(cosine_similarity)
        bin_assignments.append(assigned_bin)

    # Organize trials by their assigned bin and then sort them by total reward
    total_rewards = np.sum(r_arr, axis=1)
    bin_lists = [[] for _ in range(N)]
    for i, assigned_bin in enumerate(bin_assignments):
        bin_lists[assigned_bin].append(i)
    
    # Sort trials in each bin by reward and select top T trials
    top_trials = np.zeros((T, N), dtype=int)
    for idx, trial_list in enumerate(bin_lists):
        sorted_trials = sorted(trial_list, key=lambda x: total_rewards[x], reverse=True)
        top_trials[:, idx] = sorted_trials[:T]

    return top_trials

def get_top_rewarded_trials_old(dot_arr, r_arr, T, N):
    norms = np.linalg.norm(dot_arr, axis=2)
    normalized_dot_arr = dot_arr / norms[:, :, np.newaxis]
    bins = generate_bin_vectors(N)
    
    # Create an array to store the top K trials for each bin
    top_trials = np.empty((T, N), dtype=int)

    # Calculate the sum of rewards for each trial
    total_rewards = np.sum(r_arr, axis=1)

    for idx, bin_val in enumerate(bins):
        # Calculate the angle (cosine similarity) between each dot vector and the current bin vector
        cosine_similarity = normalized_dot_arr @ np.array(bin_val)
        
        # Convert cosine similarity to angular difference
        angular_diff = np.arccos(cosine_similarity)

        # Check if this bin has the smallest angular difference for each dot vector
        is_closest_bin = np.isclose(angular_diff, np.min(angular_diff, axis=1)[:, np.newaxis])
        
        # Find the top K trials for the current bin
        sorted_indices = np.argsort(total_rewards[is_closest_bin[:, 0]])[::-1]
        top_trials[:, idx] = np.arange(1000)[is_closest_bin[:, 0]][sorted_indices[:T]]

    return top_trials

def get_top_rewarded_trials(dot_arr, r_arr, T, N):
    norms = np.linalg.norm(dot_arr, axis=2)
    normalized_dot_arr = dot_arr / norms[:, :, np.newaxis]
    bins = generate_bin_vectors(N)
    
    # Store the trials for each bin in a list of lists
    bin_trials = [[] for _ in range(N)]

    for trial_idx, dots in enumerate(normalized_dot_arr):
        # Calculate the angle (cosine similarity) between this trial's dot vector and all bins
        cosine_similarity = dots @ bins.T
        angular_diff = np.arccos(cosine_similarity)
        # Get the index of the bin that has the smallest angular difference for this trial
        closest_bin_idx = np.argmin(angular_diff, axis=1)[0] 
        bin_trials[closest_bin_idx].append(trial_idx)
        
    top_trials = np.empty((T, N), dtype=int)
    total_rewards = np.sum(r_arr, axis=1)

    for idx, trials in enumerate(bin_trials):
        # Sort the trials in this bin based on their rewards and take the top T
        sorted_trials = sorted(trials, key=lambda x: total_rewards[x], reverse=True)
        top_trials[:, idx] = sorted_trials[:T]

    t = has_duplicates(top_trials)
    print('has duplicates=',t)

    return top_trials

def plot_scatter_top_trials(dot_arr, top_trials, bins):
    # Calculate the average dot velocity vectors
    dot_diffs = np.diff(dot_arr, axis=1)
    mean_dot_vel = np.mean(dot_diffs, axis=1)
    
    plt.figure(figsize=(10,10))
    
    # Draw the dotted lines for bin edges
    max_radius = np.max(np.linalg.norm(mean_dot_vel, axis=1))
    for bin_vec in bins:
        end_x = max_radius * bin_vec[0]
        end_y = max_radius * bin_vec[1]
        plt.plot([0, end_x], [0, end_y], 'k--')
    
    # Scatter plot for each bin and compute the center of mass
    for idx, trials in enumerate(top_trials.T):
        selected_dots = mean_dot_vel[trials]
        plt.scatter(selected_dots[:, 0], selected_dots[:, 1], label=f'Bin {idx}', s=20, alpha=0.5)
        
        # Calculate and plot the center of mass for this bin
        centroid_x = np.mean(selected_dots[:, 0])
        centroid_y = np.mean(selected_dots[:, 1])
        plt.scatter(centroid_x, centroid_y, marker='x', s=200)
        
    plt.legend()
    plt.title("Scatter plot of dot velocities for top trials with centroids")
    plt.xlabel("X-coordinate")
    plt.ylabel("Y-coordinate")
    plt.grid(True)
    plt.show()

N = 8
T = 100

bins = generate_bin_vectors(N)  # Ensure you have the bins for your problem

# Generate top trials using the old function
top_trials = get_top_rewarded_trials_new(dot_arr, r_arr, T, N)
print('top_trials=',top_trials.shape)

# Plot the scatter plot using the generated top trials
plot_scatter_top_trials(dot_arr, top_trials, bins)


In [ ]:
# debug (dot vel scatter)

%matplotlib inline
dot_diffs = np.diff(dot_arr, axis=1)
mean_dot_vel = np.mean(dot_diffs, axis=1)
norm_dot_vel = np.linalg.norm(mean_dot_vel, axis=1)
mean_dot_vel = mean_dot_vel / norm_dot_vel[:,np.newaxis]
print('dot_diffs.shape=',dot_diffs.shape)
print('mean_dot_vel.shape=',mean_dot_vel.shape)
print('norm_dot_vel.shape=',norm_dot_vel.shape)
print('mean_dot_vel=',mean_dot_vel.shape)
# plot a scatter plot of the dot velocities:
plt.figure(figsize=(10,6))
plt.scatter(mean_dot_vel[:,0], mean_dot_vel[:,1], s=5)
plt.title("Scatter Plot of Dot Velocities")
plt.xlabel("Dot Velocity in X")
plt.ylabel("Dot Velocity in Y")

In [ ]:
# get_top_rewarded_trials fncs (old)

def get_top_rewarded_trials_old(dot_arr, r_arr, T, N):
    norms = np.linalg.norm(dot_arr, axis=2)
    normalized_dot_arr = dot_arr / norms[:, :, np.newaxis]
    bins = generate_bin_vectors(N)
    print('bins=',bins)
    
    # Create an array to store the top K trials for each bin
    top_trials = np.empty((T, N), dtype=int)

    # Calculate the sum of rewards for each trial
    total_rewards = np.sum(r_arr, axis=1)

    for idx, bin_val in enumerate(bins):
        print('bin_val=',bin_val)
        # Calculate the angle (cosine similarity) between each dot vector and the current bin vector
        cosine_similarity = normalized_dot_arr @ np.array(bin_val)
        
        # Convert cosine similarity to angular difference
        angular_diff = np.arccos(cosine_similarity)
        print('angular_diff=',angular_diff.shape)

        # print('min_ang_diff=',np.min(angular_diff, axis=1))

        # Check if this bin has the smallest angular difference for each dot vector
        is_closest_bin = np.isclose(angular_diff, np.min(angular_diff, axis=1)[:, np.newaxis])
        
        print(f"Number of trials closest to bin {idx}: {sum(is_closest_bin[:, 0])}")
        # Find the top K trials for the current bin
        sorted_indices = np.argsort(total_rewards[is_closest_bin[:, 0]])[::-1]
        top_trials[:, idx] = np.arange(1000)[is_closest_bin[:, 0]][sorted_indices[:T]]

    t = has_duplicates(top_trials)
    print('has duplicates=',t)

    return top_trials

def get_top_rewarded_trials(dot_arr, r_arr, T, N):
    norms = np.linalg.norm(dot_arr, axis=2)
    normalized_dot_arr = dot_arr / norms[:, :, np.newaxis]
    bins = generate_bin_vectors(N)
    
    # Store the trials for each bin in a list of lists
    bin_trials = [[] for _ in range(N)]

    for trial_idx, dots in enumerate(normalized_dot_arr):
        # Calculate the angle (cosine similarity) between this trial's dot vector and all bins
        cosine_similarity = dots @ bins.T
        angular_diff = np.arccos(cosine_similarity)
        # Get the index of the bin that has the smallest angular difference for this trial
        closest_bin_idx = np.argmin(angular_diff, axis=1)[0] 
        bin_trials[closest_bin_idx].append(trial_idx)
        
    top_trials = np.empty((T, N), dtype=int)
    total_rewards = np.sum(r_arr, axis=1)

    for idx, trials in enumerate(bin_trials):
        # Sort the trials in this bin based on their rewards and take the top T
        sorted_trials = sorted(trials, key=lambda x: total_rewards[x], reverse=True)
        top_trials[:, idx] = sorted_trials[:T]

    t = has_duplicates(top_trials)
    print('has duplicates=',t)

    return top_trials

In [ ]:
# old

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.collections import LineCollection

n = 335

def plot_trajectory(data, n):
    fig = plt.figure(figsize=(12, 6))
    
    # 3D Plot
    ax1 = fig.add_subplot(121, projection='3d')
    traj = data[n, :, :3]  # Considering top 3 PCs
    length = traj.shape[0]
    segments = [traj[i:i+2] for i in range(length-1)]
    lc_3d = Line3DCollection(segments, cmap=plt.cm.seismic, norm=plt.Normalize(0, length))
    lc_3d.set_array(np.linspace(0, length, len(segments)))
    ax1.add_collection(lc_3d)
    ax1.set_xlim(traj[:, 0].min(), traj[:, 0].max())
    ax1.set_ylim(traj[:, 1].min(), traj[:, 1].max())
    ax1.set_zlim(traj[:, 2].min(), traj[:, 2].max())
    ax1.set_title('3D Trajectory')
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    ax1.set_zlabel('PC3')

    # 2D Plot (assuming the plane of most variation involves the first and second PCs)
    ax2 = fig.add_subplot(122)
    segments_2d = [traj[i:i+2, :2] for i in range(length-1)]
    lc_2d = LineCollection(segments_2d, cmap=plt.cm.seismic, norm=plt.Normalize(0, length))
    lc_2d.set_array(np.linspace(0, length, len(segments_2d)))
    ax2.add_collection(lc_2d)
    ax2.autoscale_view()
    ax2.set_title('2D Projection')
    ax2.set_xlabel('PC1')
    ax2.set_ylabel('PC2')

    plt.tight_layout()
    plt.show()
    plt.close()

plot_trajectory(hs_pc_s, n)
# plot_trajectory(hv_pc_c, n)



In [ ]:
# ax
ax1.set_xlabel('PC1', fontsize=fnt, labelpad=-5, rotation=0)
    ax1.set_xticks([])
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['left'].set_linewidth(4)
    ax1.spines['bottom'].set_linewidth(4)
    # ax1.xaxis.pane.fill = False
    ax1.set_ylabel('PC2', fontsize=fnt, labelpad=-0, rotation=0)
    ax1.set_yticks([])
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['left'].set_linewidth(4)
    ax1.spines['bottom'].set_linewidth(4)
    ax1.set_zlabel('PC3', fontsize=fnt, labelpad=-5, rotation=90)
    ax1.set_zticks([])
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['left'].set_linewidth(4)
    ax1.spines['bottom'].set_linewidth(4)
    ax1.grid(False)

    ax1.legend(handles=legend_elements, loc="upper center", ncol=1, fontsize=15, frameon=False, handlelength=1.4, columnspacing=0.5, bbox_to_anchor=(0.1, .9))